In [1]:
!pip install gurobipy

In [2]:
params = {
"WLSACCESSID": '3bb2aa05-fbf2-40de-9010-2927967a3661',
"WLSSECRET": 'fcc26b83-866c-418a-971d-650e68870b36',
"LICENSEID": 2652745,
"Threads": 95,
"NumericFocus":0,
}


In [37]:
import gurobipy as gp
from gurobipy import GRB
from typing import List, Dict, Tuple, Optional, Any, Set # Added Set for Label

# Placeholder for Label class and ESPPRC functions if not defined elsewhere
# For this snippet to be runnable, you'd need:
# - Label class definition
# - initialize_depot_label(data, is_forward)
# - solve_espprc_forward_labeling(data, dual_prices, alpha_penalty, depot_return_time_limit)
# - solve_espprc_bidirectional(data, dual_prices, alpha_penalty) -> though current wrapper uses forward
# - calculate_distance_matrix(coordinates_data)
# - calculate_travel_time_matrix(distances, speed)
# These would typically be in the same file or imported.

def solve_restricted_master_problem(current_routes: List[Dict[str, Any]],
                                    customers_list: List[int],
                                    alpha_penalty: float,
                                    model_name: str = "RMP") -> Tuple[Optional[gp.Model], Optional[Dict[int, float]]]:
    """
    Solves the LP relaxation of the Restricted Master Problem.

    Args:
        current_routes: List of route dictionaries. Each route dict should contain:
                        'id': A unique route identifier.
                        'cost': The travel distance of the route.
                        'customers_served': A list of customer IDs served by this route.
        customers_list: List of all customer IDs.
        alpha_penalty: Penalty for using a vehicle/route.
        model_name: Name for the Gurobi model.

    Returns:
        A tuple (model, dual_prices). dual_prices is a dict {customer_id: dual_value}.
        Returns (None, None) if solving fails.
    """
    env = gp.Env(params=params)
    rmp = gp.Model(model_name, env = env)
    #rmp.setParam('OutputFlag', 0)  # Suppress Gurobi output

    route_vars: Dict[str, gp.Var] = {}

    # Define variables and objective
    for idx, route_details in enumerate(current_routes):
        route_id = route_details.get('id', f"route_{idx}")
        # Cost C_r = (travel distance of route r) + alpha_penalty
        cost_r = route_details['cost'] + alpha_penalty
        route_vars[route_id] = rmp.addVar(obj=cost_r, vtype=GRB.CONTINUOUS, lb=0.0, name=f"lambda_{route_id}")

    rmp.modelSense = GRB.MINIMIZE

    # Define constraints (Set Covering: sum(a_ir * lambda_r) >= 1 for each customer i)
    customer_constraints: Dict[int, gp.Constr] = {}
    for i in customers_list:
        # a_ir is 1 if customer i is in route_details['customers_served']
        expr = gp.quicksum(route_vars[r_details.get('id', f"route_{r_idx}")]
                           for r_idx, r_details in enumerate(current_routes)
                           if i in r_details['customers_served'])

        # Only add constraint if the customer is actually served by at least one route in the current pool
        # This avoids issues with empty expressions if a customer is not yet covered.
        if expr.size() > 0:
            customer_constraints[i] = rmp.addConstr(expr >= 1, name=f"cover_cust_{i}")
        else:
            # If a customer is not covered by any route in the pool, its dual price is effectively
            # what the ESPPRC needs to overcome. Gurobi won't assign a .Pi to a non-existent constraint.
            # We'll handle this when constructing dual_prices.
            pass


    try:
        rmp.optimize()
    except gp.GurobiError as e:
        print(f"Gurobi error during RMP optimization: {e}")
        return None, None

    if rmp.status == GRB.OPTIMAL:
        dual_prices: Dict[int, float] = {}
        for i in customers_list:
            if i in customer_constraints: # Constraint exists for this customer
                dual_prices[i] = customer_constraints[i].Pi
            else: # Customer not covered by any route in current RMP pool
                  # The "true" dual can be thought of as very high, or simply 0 if no constraint.
                  # ESPPRC should still try to find paths for such customers.
                  # Setting to 0 is a common practice if constraint doesn't exist.
                dual_prices[i] = 0.0
        return rmp, dual_prices
    else:
        print(f"RMP optimization failed with status: {rmp.status}")
        # Attempt to retrieve duals if model is infeasible but duals might exist (e.g. IIS)
        # This is advanced and depends on Gurobi's state. For simplicity, return None.
        return None, None


def solve_master_ip(final_routes: List[Dict[str, Any]],
                    customers_list: List[int],
                    alpha_penalty: float,
                    model_name: str = "MasterIP") -> Tuple[Optional[gp.Model], Optional[List[Dict[str, Any]]]]:
    """
    Solves the Master Problem as an Integer Program with the final set of routes.
    """
    # Create a new Gurobi environment and model
    # Using default environment is usually fine unless specific parameters are needed globally
    env = gp.Env(params=params)
    master_ip = gp.Model(model_name)
    #master_ip.setParam('OutputFlag', 0) # Suppress Gurobi output by default

    route_vars: Dict[str, gp.Var] = {}

    if not final_routes:
        print("Master IP warning: No routes provided in final_routes pool.")
        return master_ip, [] # Return empty solution

    for idx, route_details in enumerate(final_routes):
        route_id = route_details.get('id', f"route_{idx}")
        cost_r = route_details['cost'] + alpha_penalty
        route_vars[route_id] = master_ip.addVar(obj=cost_r, vtype=GRB.BINARY, name=f"lambda_{route_id}")

    master_ip.modelSense = GRB.MINIMIZE

    # Ensure all customers can be covered by at least one route in the pool
    for i in customers_list:
        expr = gp.quicksum(route_vars[r_details.get('id', f"route_{r_idx}")]
                           for r_idx, r_details in enumerate(final_routes)
                           if i in r_details['customers_served'])
        if expr.size() > 0:
            master_ip.addConstr(expr >= 1, name=f"cover_cust_{i}")
        else:
            # This customer cannot be served by any route in the final pool.
            # The IP will be infeasible. This indicates an issue in the column generation.
            print(f"ERROR in Master IP: Customer {i} cannot be served by any route in the final pool. IP will likely be infeasible.")
            # To allow the model to solve for other customers, you might skip this constraint,
            # or the problem is ill-defined if a customer has no routes.
            # For now, we add it, which will lead to infeasibility if expr is empty.
            master_ip.addConstr(expr >= 1, name=f"cover_cust_{i}") # This will make it infeasible if expr is empty.

    try:
        master_ip.optimize()
    except gp.GurobiError as e:
        print(f"Gurobi error during Master IP optimization: {e}")
        return master_ip, None # Return model for inspection

    selected_routes_details: List[Dict[str, Any]] = []
    if master_ip.status == GRB.OPTIMAL or \
       (master_ip.status in [GRB.TIME_LIMIT, GRB.INTERRUPTED, GRB.SUBOPTIMAL] and master_ip.SolCount > 0):

        if master_ip.SolCount > 0: # Check if a solution was actually found
            print(f"Master IP Objective: {master_ip.ObjVal}")
            for idx, route_details in enumerate(final_routes):
                route_id = route_details.get('id', f"route_{idx}")
                if route_vars[route_id].X > 0.5: # Check if variable is selected
                    selected_routes_details.append(route_details)
            print(f"Number of vehicles/routes used: {len(selected_routes_details)}")
        else: # e.g. Time limit hit before any feasible solution found
            print("Master IP optimization ended, but no feasible integer solution found within limits.")
            return master_ip, None

    elif master_ip.status == GRB.INFEASIBLE:
        print("Master IP is INFEASIBLE. Check if all customers can be covered by the generated routes.")
        # You might want to compute an IIS here to debug
        # master_ip.computeIIS()
        # master_ip.write("master_ip_infeasible.ilp")
        return master_ip, None
    else:
        print(f"Master IP did not solve to optimality or find a feasible solution. Status: {master_ip.status}")
        return master_ip, None

    return master_ip, selected_routes_details


def solve_espprc_with_multiple_time_windows(
    customers_list_espprc: List[int],
    demands_espprc: Dict[int, float],
    service_times_espprc: Dict[int, float],
    time_windows_data_espprc: Dict[int, List[Tuple[int, int]]],
    coordinates_data_espprc: Dict[int, Tuple[float, float]], # Used if matrices not pre-calculated
    vehicle_capacity_espprc: float,
    speed_espprc: float, # Expected in dist/hr
    alpha_penalty_espprc: float,
    dual_prices_espprc: Dict[int, float],
    depot_id_espprc: int = 0,
    depot_return_time_limit_espprc: float = float('inf'),
    # Optional pre-calculated matrices to save computation
    travel_times_matrix: Optional[Dict[Tuple[int, int], float]] = None,
    travel_distances_matrix: Optional[Dict[Tuple[int, int], float]] = None
) -> Tuple[Optional[Dict[str, Any]], float]:
    """
    Wrapper for ESPPRC solver.
    Constructs the 'data' dictionary and calls the chosen ESPPRC algorithm.
    Currently set to use forward labeling.
    """

    # Calculate matrices if not provided
    if travel_distances_matrix is None:
        # Need calculate_distance_matrix function defined
        travel_distances_matrix = calculate_distance_matrix(coordinates_data_espprc)
    if travel_times_matrix is None:
        # Need calculate_travel_time_matrix function defined
        travel_times_matrix = calculate_travel_time_matrix(travel_distances_matrix, speed_espprc)

    data_for_espprc = {
        'customers_list': customers_list_espprc,
        'demands': demands_espprc,
        'service_times': service_times_espprc,
        'time_windows_data': time_windows_data_espprc,
        'coordinates_data': coordinates_data_espprc, # For reference, though matrices are used
        'vehicle_capacity': vehicle_capacity_espprc,
        'speed': speed_espprc, # dist/hr, used by calculate_travel_time_matrix if needed
        'depot_id': depot_id_espprc,
        'depot_info': {
            'id': depot_id_espprc,
            'time_window': (0, depot_return_time_limit_espprc), # ESPPRC needs depot's operational window end
            'demand': 0.0,
            'service_time': 0.0
        },
        # dual_prices are passed directly to the ESPPRC solver, not typically via this data dict
        'travel_times': travel_times_matrix,
        'travel_distances': travel_distances_matrix
    }

    # --- Choose ESPPRC Algorithm ---
    # For now, using forward labeling as per previous setup.
    # Replace with solve_espprc_bidirectional if/when it's fully implemented.
    # Ensure the chosen ESPPRC function (e.g., solve_espprc_forward_labeling) is defined.

    # Example: Assuming solve_espprc_forward_labeling is the one to use
    # This function needs to be defined elsewhere in your script.
    # from your_module import solve_espprc_forward_labeling

    newly_found_routes = solve_espprc_forward_labeling(
        data_for_espprc,
        dual_prices_espprc,
        alpha_penalty_espprc,
        depot_return_time_limit_espprc # Pass the specific limit for depot return
    )

    if newly_found_routes:
        # ESPPRC should return routes sorted by reduced cost, or we sort here.
        # Assuming it returns a list of route dicts, each with 'reduced_cost'.
        best_new_route = min(newly_found_routes, key=lambda x: x['reduced_cost'])
        return best_new_route, best_new_route['reduced_cost']
    else:
        return None, float('inf')


def column_generation_solver(initial_routes: List[Dict[str, Any]],
                             customers_list: List[int],
                             demands: Dict[int, float],
                             service_times: Dict[int, float],
                             time_windows_data: Dict[int, List[Tuple[int, int]]],
                             coordinates_data: Dict[int, Tuple[float, float]],
                             vehicle_capacity: float,
                             speed: float,  # Expected in dist/hr
                             alpha_penalty: float,
                             max_cg_iterations: int = 50,
                             # Added parameters for depot information:
                             depot_id_cg: int = 1,
                             depot_max_time_cg: float = 1440.0  # Default to 24 hours in minutes
                            ) -> Tuple[Optional[gp.Model], Optional[List[Dict[str, Any]]], List[Dict[str, Any]]]:
    """
    Main column generation loop.

    Args:
        ... (standard VRP data) ...
        depot_id_cg: The ID of the depot node.
        depot_max_time_cg: The latest time a vehicle can return to the depot (minutes from midnight).

    Returns:
        (final_ip_model, selected_routes_list, all_routes_in_pool_list)
    """
    current_routes_pool = list(initial_routes)

    # Pre-calculate travel times and distances once for the entire CG process
    # These are based on the full coordinates_data and speed.
    # Ensure these helper functions are defined in your environment.
    print("CG: Pre-calculating distance and travel time matrices...")
    cg_travel_distances_matrix = calculate_distance_matrix(coordinates_data)
    cg_travel_times_matrix = calculate_travel_time_matrix(cg_travel_distances_matrix, speed)
    print("CG: Matrices calculated.")

    last_rmp_obj_val = float('inf')

    for iteration in range(max_cg_iterations):
        print(f"\n--- Column Generation Iteration: {iteration + 1} ---")
        print(f"Current number of routes in RMP: {len(current_routes_pool)}")

        if not current_routes_pool:
            print("Error in CG: Route pool is empty. Cannot solve RMP. This might happen if initial routes are all invalid.")
            # Attempt to generate at least one route if possible, e.g., for each customer if feasible
            # This is a recovery mechanism, ideally initial_routes should be valid.
            print("Attempting to generate basic single-customer routes for ESPPRC...")
            temp_dual_prices = {cust_id: 1000 for cust_id in customers_list} # High artificial duals

            # Call ESPPRC with artificial duals to try to get *any* feasible route
            # This call to ESPPRC is to bootstrap if the pool is empty.
            # It might be better to ensure initial_routes are valid from the start.
            bootstrap_route, bootstrap_rc = solve_espprc_with_multiple_time_windows(
                customers_list_espprc=customers_list, demands_espprc=demands,
                service_times_espprc=service_times, time_windows_data_espprc=time_windows_data,
                coordinates_data_espprc=coordinates_data, vehicle_capacity_espprc=vehicle_capacity,
                speed_espprc=speed, alpha_penalty_espprc=alpha_penalty,
                dual_prices_espprc=temp_dual_prices, depot_id_espprc=depot_id_cg,
                depot_return_time_limit_espprc=depot_max_time_cg,
                travel_times_matrix=cg_travel_times_matrix,
                travel_distances_matrix=cg_travel_distances_matrix
            )
            if bootstrap_route:
                print(f"Bootstrap: Found an initial route: {bootstrap_route['id']}")
                current_routes_pool.append(bootstrap_route)
            else:
                print("Bootstrap failed: Could not generate any initial route via ESPPRC.")
                return None, None, current_routes_pool


        rmp_model, dual_prices = solve_restricted_master_problem(
            current_routes_pool, customers_list, alpha_penalty
        )

        if rmp_model is None or dual_prices is None:
            print("CG: Failed to solve RMP. Stopping column generation.")
            break

        current_rmp_obj = rmp_model.ObjVal
        print(f"RMP Objective Value: {current_rmp_obj:.2f}")
        # print(f"Dual prices from RMP: {dual_prices}") # For debugging

        # Solve the Pricing Subproblem (ESPPRC)
        best_new_route, reduced_cost = solve_espprc_with_multiple_time_windows(
            customers_list_espprc=customers_list,
            demands_espprc=demands,
            service_times_espprc=service_times,
            time_windows_data_espprc=time_windows_data,
            coordinates_data_espprc=coordinates_data, # Passed for completeness, matrices are used
            vehicle_capacity_espprc=vehicle_capacity,
            speed_espprc=speed, # dist/hr
            alpha_penalty_espprc=alpha_penalty,
            dual_prices_espprc=dual_prices,
            depot_id_espprc=depot_id_cg, # Use the passed depot ID
            depot_return_time_limit_espprc=depot_max_time_cg, # Use the passed depot max time
            travel_times_matrix=cg_travel_times_matrix, # Pass pre-calculated matrix
            travel_distances_matrix=cg_travel_distances_matrix # Pass pre-calculated matrix
        )

        if best_new_route and reduced_cost < -1e-6:  # Threshold for negative reduced cost
            print(f"Found new route with ID {best_new_route.get('id', 'N/A')}, reduced cost: {reduced_cost:.2f}")
            # print(f"Route details: Customers {best_new_route['customers_served']}, Cost {best_new_route['cost']:.2f}")

            # Basic duplicate check (customers and cost)
            is_duplicate = False
            for r_in_pool in current_routes_pool:
                if set(r_in_pool['customers_served']) == set(best_new_route['customers_served']) and \
                   abs(r_in_pool['cost'] - best_new_route['cost']) < 1e-6:
                    is_duplicate = True
                    # print("New route is considered a duplicate of an existing route in the pool.")
                    break

            if not is_duplicate:
                current_routes_pool.append(best_new_route)
            else:
                # If it's a duplicate but still has the most negative reduced cost,
                # it might mean the LP is optimal or cycling.
                # A more robust check for stopping might be if reduced_cost is very close to zero
                # for several iterations or if RMP objective doesn't improve.
                print("New route is a duplicate. If reduced cost is still significantly negative, consider alternative stopping criteria.")
                if reduced_cost > -0.01: # If not significantly negative, likely near optimal
                    print("Reduced cost close to zero, stopping CG.")
                    break
                # else continue, maybe it helps break cycling or explore other parts of solution space
        else:
            print("No new route with sufficiently negative reduced cost found. LP likely optimal for current columns.")
            break # Exit loop if pricing problem proves LP optimality for the current set of columns

        # Optional: Check for RMP objective stalling
        if abs(current_rmp_obj - last_rmp_obj_val) < 1e-4 and iteration > 5: # Arbitrary stall check
            print("RMP objective has stalled for several iterations. Stopping CG.")
            break
        last_rmp_obj_val = current_rmp_obj


    print("\n--- Column Generation Finished ---")
    print(f"Final number of routes generated: {len(current_routes_pool)}")

    if not current_routes_pool:
        print("CG Error: No routes in the pool to solve the Master IP.")
        return None, None, []

    print("\nSolving final Master Problem as IP...")
    final_ip_model, selected_routes = solve_master_ip(
        current_routes_pool, customers_list, alpha_penalty
    )

    if selected_routes is not None: # Check if a solution (even if empty) was returned
        # (Result processing happens in the main script)
        return final_ip_model, selected_routes, current_routes_pool
    else: # Master IP failed or returned None for selected_routes
        print("No final integer solution found or Master IP failed.")
        return final_ip_model, None, current_routes_pool

In [4]:
import math
from datetime import datetime, timedelta

def time_to_minutes(time_str):
    """Convert time string 'HH:MM' to minutes from midnight."""
    hours, minutes = map(int, time_str.split(':'))
    return hours * 60 + minutes

def load_coordinates(filepath):
    """Load coordinates from instance_coordinates.txt"""
    coordinates = {}
    with open(filepath, 'r') as f:
        for line in f:
            parts = line.strip().split()
            if len(parts) == 3:
                node_id = int(parts[0])
                x = float(parts[1])
                y = float(parts[2])
                coordinates[node_id] = (x, y)
    return coordinates

def load_demands(filepath):
    """Load demands from instance_demand.txt"""
    demands = {}
    with open(filepath, 'r') as f:
        for line in f:
            parts = line.strip().split()
            if len(parts) == 2:
                customer_id = int(parts[0])
                demand = float(parts[1])
                demands[customer_id] = demand
    return demands

def load_service_times(filepath):
    """Load service times from instance_service_time.txt"""
    service_times = {}
    with open(filepath, 'r') as f:
        for line in f:
            parts = line.strip().split()
            if len(parts) == 2:
                customer_id = int(parts[0])
                service_time = float(parts[1])  # in minutes
                service_times[customer_id] = service_time
    return service_times

def load_time_windows(filepath):
    """Load time windows from instance_time_windows.txt"""
    time_windows = {}
    with open(filepath, 'r') as f:
        for line in f:
            parts = line.strip().split()
            if len(parts) == 3:
                customer_id = int(parts[0])
                start_time = time_to_minutes(parts[1])
                end_time = time_to_minutes(parts[2])

                if customer_id not in time_windows:
                    time_windows[customer_id] = []
                time_windows[customer_id].append((start_time, end_time))

    # Sort time windows for each customer
    for customer_id in time_windows:
        time_windows[customer_id].sort()

    return time_windows

def calculate_distance_matrix(coordinates):
    """Calculate Euclidean distances between all pairs of locations"""
    distances = {}
    for i in coordinates:
        for j in coordinates:
            if i != j:
                x1, y1 = coordinates[i]
                x2, y2 = coordinates[j]
                dist = math.sqrt((x2 - x1)**2 + (y2 - y1)**2)
                distances[(i, j)] = dist
            else:
                distances[(i, j)] = 0.0
    return distances

def calculate_travel_time_matrix(distances, speed=1000):
    """Calculate travel times from distances and speed"""
    travel_times = {}
    for (i, j), distance in distances.items():
        if i != j:
            # Add 1 for triangle inequality as mentioned in the paper
            travel_time = distance / speed  # in hours
            travel_times[(i, j)] = travel_time * 60  # convert to minutes
        else:
            travel_times[(i, j)] = 0.0
    return travel_times

def setup_depot_info(depot_id=1, planning_horizon_hours=24):
    """Setup depot information"""
    return {
        'id': depot_id,
        'time_window': (0, planning_horizon_hours * 60),  # 24 hours in minutes
        'demand': 0,
        'service_time': 0
    }

def load_and_preprocess_data(base_path, instance_name, speed=1000, vehicle_capacity=12600, depot_id=1):
    """
    Load all instance data and perform preprocessing for VRPMTW.

    Args:
        base_path (str): Path to directory containing instance files
        instance_name (str): Name prefix of instance files (e.g., 'instance' for instance_*.txt)
        speed (float): Vehicle speed in distance units per hour (default: 1000)
        vehicle_capacity (float): Vehicle capacity in kg (default: 12600)
        depot_id (int): ID of the depot node (default: 1)

    Returns:
        dict: Dictionary containing all preprocessed data with the following keys:
            - 'customers_list' (list): List of customer IDs (excludes depot)
            - 'coordinates_data' (dict): {node_id: (x, y)} coordinates for all nodes
            - 'demands' (dict): {customer_id: demand_kg} customer demands in kg
            - 'service_times' (dict): {customer_id: service_time_minutes} service times
            - 'time_windows_data' (dict): {customer_id: [(start_min, end_min), ...]}
                                         multiple time windows per customer in minutes from midnight
            - 'travel_times' (dict): {(i,j): travel_time_minutes} travel times between all node pairs
            - 'travel_distances' (dict): {(i,j): euclidean_distance} distances between all node pairs
            - 'depot_info' (dict): {'id': depot_id, 'time_window': (0, 1440), 'demand': 0, 'service_time': 0}
            - 'vehicle_capacity' (float): Vehicle capacity in kg
            - 'speed' (float): Vehicle speed in distance units per hour
            - 'depot_id' (int): ID of the depot node

    Example:
        data = load_and_preprocess_data("/content", "instance")
        customers = data['customers_list']  # [2, 3, 4, 5, ...]
        demand_customer_2 = data['demands'][2]  # 2290.0
        travel_time_1_to_2 = data['travel_times'][(1, 2)]  # 15.4 minutes
    """
    print("Loading instance data...")

    # Construct file paths
    coords_file = f"{base_path}/{instance_name}_coordinates.txt"
    demand_file = f"{base_path}/{instance_name}_demand.txt"
    service_file = f"{base_path}/{instance_name}_service_time.txt"
    tw_file = f"{base_path}/{instance_name}_time_windows.txt"

    # Load raw data
    coordinates = load_coordinates(coords_file)
    demands = load_demands(demand_file)
    service_times = load_service_times(service_file)
    time_windows = load_time_windows(tw_file)

    # Set depot service time to 0 if not specified
    if depot_id not in service_times:
        service_times[depot_id] = 0

    # Set depot time window (24 hours)
    time_windows[depot_id] = [(0, 24 * 60)]  # 0 to 1440 minutes

    # Calculate distance and travel time matrices
    print("Calculating distance and travel time matrices...")
    distances = calculate_distance_matrix(coordinates)
    travel_times = calculate_travel_time_matrix(distances, speed)

    # Create customer list (exclude depot)
    customers_list = [node_id for node_id in coordinates.keys() if node_id != depot_id]

    # Setup depot info
    depot_info = setup_depot_info(depot_id)

    print(f"Loaded {len(customers_list)} customers")
    print(f"Depot: {depot_id}")
    print(f"Vehicle capacity: {vehicle_capacity} kg")
    print(f"Speed: {speed} distance units/hour")

    return {
        'customers_list': customers_list,
        'coordinates_data': coordinates,
        'demands': demands,
        'service_times': service_times,
        'time_windows_data': time_windows,
        'travel_times': travel_times,
        'travel_distances': distances,
        'depot_info': depot_info,
        'vehicle_capacity': vehicle_capacity,
        'speed': speed,
        'depot_id': depot_id
    }

# Test function to validate data loading
def print_data_summary(data):
    """Print a summary of loaded data for validation"""
    print("\n=== DATA SUMMARY ===")
    print(f"Number of customers: {len(data['customers_list'])}")
    print(f"Depot ID: {data['depot_info']['id']}")
    print(f"Vehicle capacity: {data['vehicle_capacity']} kg")

    # Sample customer info
    sample_customer = data['customers_list'][0] if data['customers_list'] else None
    if sample_customer:
        print(f"\nSample customer {sample_customer}:")
        print(f"  Coordinates: {data['coordinates_data'][sample_customer]}")
        print(f"  Demand: {data['demands'][sample_customer]} kg")
        print(f"  Service time: {data['service_times'].get(sample_customer, 'N/A')} min")
        print(f"  Time windows: {data['time_windows_data'].get(sample_customer, 'N/A')}")

    # Check for customers with multiple time windows
    multi_tw_customers = []
    for cust in data['customers_list']:
        if cust in data['time_windows_data'] and len(data['time_windows_data'][cust]) > 1:
            multi_tw_customers.append(cust)

    if multi_tw_customers:
        print(f"\nCustomers with multiple time windows: {multi_tw_customers}")
        for cust in multi_tw_customers[:3]:  # Show first 3
            print(f"  Customer {cust}: {data['time_windows_data'][cust]}")

    print("===================\n")

In [5]:
def evaluate_route(route_sequence, departure_time, data):
    """
    Evaluate a potential route for feasibility and calculate total travel distance.

    Service can span multiple time windows: if service time extends beyond the current
    window, the driver pauses and waits for the next window to resume service.

    Args:
        route_sequence (list): Ordered list of customer IDs to visit
        departure_time (float): Departure time from depot in minutes from midnight
        data (dict): Preprocessed data dictionary

    Returns:
        float or None: Total travel distance if feasible, None if infeasible
    """
    if not route_sequence:
        return 0.0  # Empty route is valid with zero distance

    depot_id = data['depot_id']
    vehicle_capacity = data['vehicle_capacity']
    travel_times = data['travel_times']
    travel_distances = data['travel_distances']
    demands = data['demands']
    service_times = data['service_times']
    time_windows_data = data['time_windows_data']
    depot_time_window = data['depot_info']['time_window']

    # Initialize route state
    current_time = departure_time
    current_load = 0.0
    current_location = depot_id
    total_distance = 0.0

    # Check if departure time is within depot's operational window
    if current_time < depot_time_window[0] or current_time > depot_time_window[1]:
        return None

    # Visit each customer in sequence
    for customer_id in route_sequence:
        # Check capacity constraint
        customer_demand = demands[customer_id]
        if current_load + customer_demand > vehicle_capacity:
            return None  # Capacity exceeded

        # Calculate travel to customer
        travel_time = travel_times[(current_location, customer_id)]
        travel_distance = travel_distances[(current_location, customer_id)]
        arrival_time = current_time + travel_time

        # Get customer's time windows and service time
        customer_time_windows = time_windows_data[customer_id]
        total_service_time = service_times[customer_id]

        # Sort time windows by start time
        sorted_windows = sorted(customer_time_windows, key=lambda x: x[0])

        # Find first window where service can start (arrival_time <= window_end)
        service_start_window_idx = None
        for i, (window_start, window_end) in enumerate(sorted_windows):
            if arrival_time <= window_end:
                service_start_window_idx = i
                break

        if service_start_window_idx is None:
            return None  # Cannot start service in any window

        # Simulate service across potentially multiple windows
        remaining_service_time = total_service_time
        window_idx = service_start_window_idx
        current_service_time = arrival_time

        while remaining_service_time > 0:
            if window_idx >= len(sorted_windows):
                return None  # No more windows available

            window_start, window_end = sorted_windows[window_idx]

            # Determine when service can actually start in this window
            service_start_in_window = max(current_service_time, window_start)

            # If we've already passed this window, move to next
            if service_start_in_window >= window_end:
                window_idx += 1
                continue

            # Calculate how much service can be done in this window
            available_time_in_window = window_end - service_start_in_window
            service_in_this_window = min(remaining_service_time, available_time_in_window)

            # Update service progress
            remaining_service_time -= service_in_this_window
            current_service_time = service_start_in_window + service_in_this_window

            # If service is complete, break
            if remaining_service_time <= 0:
                break

            # Need to continue in next window - jump to next window start
            window_idx += 1
            if window_idx < len(sorted_windows):
                current_service_time = sorted_windows[window_idx][0]

        if remaining_service_time > 0:
            return None  # Service could not be completed within available windows

        # Update route state
        current_time = current_service_time
        current_load += customer_demand
        current_location = customer_id
        total_distance += travel_distance

    # Check if can return to depot within depot's time window
    if current_location != depot_id:  # Need to return to depot
        return_travel_time = travel_times[(current_location, depot_id)]
        return_distance = travel_distances[(current_location, depot_id)]
        depot_arrival_time = current_time + return_travel_time

        if depot_arrival_time > depot_time_window[1]:
            return None  # Cannot return to depot in time

        total_distance += return_distance

    return total_distance

In [6]:
class Label:
    """
    Represents a label for a partial path in the ESPPRC (both forward and backward).

    Attributes:
        current_node_id (int): The last/first customer visited (forward/backward).
        visited_customers_set (set): Set of customer IDs already included in this partial path.
        current_load (float): Total demand accumulated so far.
        dominant_intervals (list): List of tuples (earliest_start, latest_start, accumulated_distance).
            For forward: dominant forward start intervals at current_node_id
            For backward: dominant backward start intervals at current_node_id
        accumulated_dual_value_sum (float): Sum of dual variables (pi_i) of visited customers.
        path_sequence (list): Ordered list of customer IDs visited in this partial path.
        is_forward (bool): True for forward label, False for backward label.
    """
    def __init__(self,
                 current_node_id: int,
                 visited_customers_set: set,
                 current_load: float,
                 dominant_forward_start_intervals: list,
                 accumulated_dual_value_sum: float,
                 path_sequence: list,
                 is_forward: bool = True):
        self.current_node_id = current_node_id
        self.visited_customers_set = set(visited_customers_set)
        self.current_load = current_load
        self.dominant_forward_start_intervals = sorted(dominant_forward_start_intervals, key=lambda x: x[0])
        self.accumulated_dual_value_sum = accumulated_dual_value_sum
        self.path_sequence = list(path_sequence)
        self.is_forward = is_forward

    def get_earliest_start_of_first_interval(self) -> float:
        """Returns E_Lf(1) for forward or E_Lb(1) for backward labels."""
        if not self.dominant_forward_start_intervals:
            return float('inf')
        return self.dominant_forward_start_intervals[0][0]

    def get_latest_start_of_last_interval(self) -> float:
        """Returns L_Lf(|F_Lf|) for forward or L_Lb(|B_Lb|) for backward labels."""
        if not self.dominant_forward_start_intervals:
            return -float('inf')
        return self.dominant_forward_start_intervals[-1][1]

    def get_num_intervals(self) -> int:
        """Returns number of dominant intervals."""
        return len(self.dominant_forward_start_intervals)

    def __repr__(self):
        direction = "Forward" if self.is_forward else "Backward"
        return (f"{direction}Label(Node: {self.current_node_id}, "
                f"Load: {self.current_load:.2f}, "
                f"Path: {self.path_sequence}, "
                f"#Intervals: {self.get_num_intervals()}, "
                f"DualSum: {self.accumulated_dual_value_sum:.2f})")

In [7]:
import math

# Assuming the Label class from Step 3 is defined above or imported
# class Label:
#     ... (implementation from Step 3) ...

def initialize_depot_label(data: dict) -> Label:
    """
    Creates the initial Label object for the depot.

    Args:
        data (dict): The preprocessed data dictionary.

    Returns:
        Label: The initial label starting at the depot.
    """
    depot_id = data['depot_info']['id']
    depot_time_window = data['depot_info']['time_window'] # (start_time, end_time)
    initial_intervals = [(depot_time_window[0], depot_time_window[1], 0.0)]

    return Label(
        current_node_id=depot_id,
        visited_customers_set=set(),
        current_load=0.0,
        dominant_forward_start_intervals=initial_intervals,
        accumulated_dual_value_sum=0.0,
        path_sequence=[depot_id]
    )

def _calculate_tau_ij_from_paper(node_i_id: int, node_j_id: int, data: dict) -> float:
    """
    Calculates tau_ij as used in the paper: service time at node i + travel time from i to j.
    Note: The paper's tau_ij definition on page 43 "Duration tau_ij includes the service
    time at node i and the travel time between nodes i and j."
    My previous _calculate_tau_ij was identical.
    """
    service_time_at_i = 0.0
    if node_i_id != data['depot_info']['id']: # If node i is not the depot
        service_time_at_i = data['service_times'].get(node_i_id, 0.0)
    else: # node_i_id is the depot
        service_time_at_i = data['depot_info']['service_time'] # This should be 0

    travel_time_ij = data['travel_times'].get((node_i_id, node_j_id), float('inf'))
    return service_time_at_i + travel_time_ij

def _generate_candidate_intervals_at_j(label_at_i: Label, customer_j_id: int, data: dict) -> list:
    """
    Generates candidate forward start intervals at customer_j_id by extending label_at_i,
    based on Algorithm C.1 from the appendix[cite: 77, 79].
    The 'cost' associated with intervals will be accumulated travel distance.
    """
    node_i_id = label_at_i.current_node_id
    generated_intervals = []

    # tau_v(Lf)j in Algorithm C.1 is service_at_v(Lf) + travel_v(Lf)_to_j
    tau_node_i_j = _calculate_tau_ij_from_paper(node_i_id, customer_j_id, data)
    travel_dist_ij = data['travel_distances'].get((node_i_id, customer_j_id), float('inf'))

    original_time_windows_at_j = data['time_windows_data'].get(customer_j_id, [])
    dominant_intervals_at_i = label_at_i.dominant_forward_start_intervals

    if node_i_id == data['depot_info']['id']: # Case: v(Lf) = 0 (depot) [cite: 77]
        # Lines 2-5 of Algorithm C.1 [cite: 77]
        # tau_0j for duration. For distance, it's data['travel_distances'][(depot_id, customer_j_id)]
        depot_travel_dist_to_j = data['travel_distances'].get((data['depot_info']['id'], customer_j_id), float('inf'))
        for e_j_t, l_j_t in original_time_windows_at_j:
            # Check direct reachability from depot based on depot's end time and tau_0j
            # Depot earliest departure is dominant_intervals_at_i[0][0] (usually 0)
            # Depot latest departure is dominant_intervals_at_i[0][1]
            depot_earliest_departure = dominant_intervals_at_i[0][0] # E_Lf(y) where y is the single depot interval

            # Effective earliest start at j if departing depot at earliest time
            earliest_service_start_at_j = max(depot_earliest_departure + tau_node_i_j, e_j_t)

            # Effective latest start at j if departing depot at latest time
            latest_service_start_at_j = min(max(dominant_intervals_at_i[0][1] + tau_node_i_j, e_j_t), l_j_t)

            if earliest_service_start_at_j <= latest_service_start_at_j :
                 generated_intervals.append(
                    (earliest_service_start_at_j, latest_service_start_at_j, depot_travel_dist_to_j)
                )
    else: # Case: v(Lf) != 0 (previous node is a customer) [cite: 77, 79]
        # Lines 7-21 of Algorithm C.1 [cite: 77, 79]
        for idx_y, y_interval in enumerate(dominant_intervals_at_i):
            E_Lf_y, L_Lf_y, D_Lf_y = y_interval # D_Lf_y is acc_dist_to_i

            for t_original_tw_j in original_time_windows_at_j:
                e_j_t, l_j_t = t_original_tw_j

                # Subcase: Interval lies before time window (Lines 9-13 of Alg C.1 [cite: 77, 79])
                if L_Lf_y + tau_node_i_j < e_j_t:
                    # Check dominance condition from Line 10 of Alg C.1 [cite: 77]
                    generates_waiting_interval = False
                    if idx_y == len(dominant_intervals_at_i) - 1: # y is the last interval F_Lf
                        generates_waiting_interval = True
                    else:
                        E_Lf_y_plus_1 = dominant_intervals_at_i[idx_y+1][0]
                        if E_Lf_y_plus_1 + tau_node_i_j > e_j_t:
                            generates_waiting_interval = True

                    if generates_waiting_interval:
                        # Add [e_j^t, e_j^t] to intervals
                        # Cost: D_Lf_y + travel_dist_ij. Waiting doesn't add to travel distance.
                        # Algorithm C.1 adds d_Lf(y) + e_j^t - L_Lf(y) to g(Lf'). This reflects waiting time in duration.
                        # For travel distance, it remains D_Lf_y + travel_dist_ij.
                        generated_intervals.append((e_j_t, e_j_t, D_Lf_y + travel_dist_ij))

                # Subcase: Overlap interval and time window (Lines 14-17 of Alg C.1 [cite: 77, 79])
                # Condition: E_Lf(y) + tau_node_i_j <= l_j_t (combined with not being strictly before)
                elif E_Lf_y + tau_node_i_j <= l_j_t: # This is the general overlap condition
                    new_E_j = max(E_Lf_y + tau_node_i_j, e_j_t)
                    new_L_j = min(L_Lf_y + tau_node_i_j, l_j_t) # Error in Alg C.1 line 15: uses min{max{L_Lf(y)...}}. Eq 3.5 is min{max{L_p(y)+tau, e_j^t}, l_j^t} for L_p'(z). Let's use 3.5 logic.
                    # Re-evaluating L_p_prime_z from eq 3.5:
                    new_L_j_corrected = min(max(L_Lf_y + tau_node_i_j, e_j_t), l_j_t)


                    if new_E_j <= new_L_j_corrected: # Valid interval
                        # Cost: D_Lf_y + travel_dist_ij. Alg C.1 (line 16) adds d_Lf(y) + tau_v(Lf),j.
                        generated_intervals.append((new_E_j, new_L_j_corrected, D_Lf_y + travel_dist_ij))

                # Break condition from Lines 18-20 of Alg C.1 [cite: 79]
                if L_Lf_y + tau_node_i_j < l_j_t: # This is L_Lf(y) in Alg C.1, not L_p_prime_z.
                                                 # If latest start from i + combined_time is before current TW end,
                                                 # it means it cannot reach later original TWs of j *any earlier*.
                                                 # This break is from the inner loop over T_j.
                    # The comment in paper [cite: 86] states "start interval y does not overlap with time windows t' > t"
                    # This implies that if L_Lf(y) + tau_v(Lf)j < l_j^t, we can break from iterating T_j FOR THE CURRENT y.
                    # And go to the next y. The code in C.1 does break.
                    pass # Python for loop will naturally go to next t_original_tw_j
                         # The break in C.1 (line 19) is for the loop over t in Tj
                         # This break should make it go to the next y interval if L_Lf(y) + tau < l_j^t
                         # This logic is subtle. The algorithm says "break", which exits the innermost loop (over T_j).
                         # So for the current 'y', if it cannot even reach the end of 'l_j_t', it surely cannot reach e_j^{t+1}.
                         # This means for the current y, we are done with time windows of j.
                    # Correction: The break should be for the inner loop over `t_original_tw_j`.
                    # If true, no need to check later time windows of j FOR THE CURRENT y_interval.
                    if L_Lf_y + tau_node_i_j < e_j_t: # If latest arrival from y is even before start of current TW_j
                                                      # (and didn't form a waiting interval)
                                                      # then it surely won't reach later TWs of j.
                        break # This break corresponds to line 19 in Alg C.1, exiting T_j loop for current y.


    return generated_intervals

def _filter_dominant_intervals_pareto(candidate_intervals: list) -> list:
    """
    Filters a list of candidate (E, L, D) intervals to keep only Pareto-dominant ones.
    An interval I1=(E1, L1, D1) dominates I2=(E2, L2, D2) if E1<=E2, L1>=L2, D1<=D2
    (and I1 != I2).
    The resulting list is sorted by E, then -L, then D.
    This function aims to produce a set that satisfies the non-overlapping property $L_k < E_{k+1}$
    implicitly through strong dominance, or by a final merging step (simplified here).
    """
    if not candidate_intervals:
        return []

    # Remove exact duplicates and sort for stable processing
    # Sort by E (asc), then L (desc for wider), then D (asc for cheaper)
    # This sorting helps in identifying dominated intervals more easily.
    unique_sorted_candidates = sorted(list(set(candidate_intervals)), key=lambda x: (x[0], -x[1], x[2]))

    dominant_intervals = []
    for current_interval in unique_sorted_candidates:
        is_dominated_by_existing = False
        # Check if current_interval is dominated by any interval already in dominant_intervals
        for existing_dominant_interval in dominant_intervals:
            # E_exist <= E_curr, L_exist >= L_curr, D_exist <= D_curr
            if (existing_dominant_interval[0] <= current_interval[0] and
                existing_dominant_interval[1] >= current_interval[1] and
                existing_dominant_interval[2] <= current_interval[2]):
                if existing_dominant_interval != current_interval: # Strictly dominated or identical covered one
                    is_dominated_by_existing = True
                    break

        if not is_dominated_by_existing:
            # Remove any intervals from dominant_intervals that are now dominated by current_interval
            new_dominant_list = []
            for i in range(len(dominant_intervals)):
                # Check if dominant_intervals[i] is dominated by current_interval
                # E_curr <= E_exist, L_curr >= L_exist, D_curr <= D_exist
                if not (current_interval[0] <= dominant_intervals[i][0] and
                        current_interval[1] >= dominant_intervals[i][1] and
                        current_interval[2] <= dominant_intervals[i][2] and
                        current_interval != dominant_intervals[i]):
                    new_dominant_list.append(dominant_intervals[i])
            dominant_intervals = new_dominant_list
            dominant_intervals.append(current_interval)
            # Re-sort is important if adding changes order for next iteration's checks,
            # but since unique_sorted_candidates is processed in order, this might be okay.
            # For safety, sort at the end.
            dominant_intervals.sort(key=lambda x: (x[0], -x[1], x[2]))


    # The list dominant_intervals now contains Pareto-optimal intervals.
    # To ensure strict non-overlapping $L_k < E_{k+1}$ as per Lemma 2.1[cite: 54]:
    # This might require a more sophisticated merging/selection if Pareto set still has overlaps.
    # For now, we return the sorted Pareto set. If overlaps cause issues later,
    # this part (non-overlapping merging) needs to be implemented robustly.
    # A simple greedy merge for same-cost adjacent/overlapping intervals:
    if not dominant_intervals:
        return []

    final_merged_intervals = []
    final_merged_intervals.append(dominant_intervals[0])

    for i in range(1, len(dominant_intervals)):
        curr_E, curr_L, curr_D = dominant_intervals[i]
        last_E, last_L, last_D = final_merged_intervals[-1]

        # If current interval is identical or strictly dominated by last_merged, it would have been filtered by Pareto.
        # Check for merging opportunity: if current starts at or before last ends, AND same cost, and extends last.
        if curr_E <= last_L + 1e-6 and curr_D == last_D : # Overlap or contiguous with same cost
            # Merge: update L of the last interval in final_merged_intervals
            final_merged_intervals[-1] = (last_E, max(last_L, curr_L), last_D)
        elif curr_E > last_L: # No overlap, current starts after last ended
            final_merged_intervals.append((curr_E, curr_L, curr_D))
        else: # Overlap with different costs, or other complex cases.
              # The Pareto filter should mean D_curr is not worse if it overlaps significantly.
              # If curr_D < last_D and they overlap, Pareto should have handled it.
              # This case means they are non-dominated but overlap.
              # Example: last=(0,10,D=5), curr=(5,15,D=5) -> merged to (0,15,D=5)
              # Example: last=(0,10,D=5), curr=(5,15,D=4) -> Pareto should keep (5,15,D=4) and (0,4.99,D=5)
              # The current simplified merge above only handles same-cost overlaps.
              # For now, if not same-cost mergeable and not starting after, just add if distinct.
              # This part is the most heuristic if not following a specific algorithm from [forthcoming].
              # Let's assume for now that after Pareto, if they are not same-cost mergeable, they form distinct dominant intervals.
              # The proof of Lemma 2.1 implies dominance sorts this out.
            if (curr_E, curr_L, curr_D) != final_merged_intervals[-1]: # Add if truly new after non-merge
                 #This could still lead to overlaps if D values are different.
                 #Safest is to return the pure Pareto set, sorted.
                 #The problem statement "dominant forward start intervals are non-overlapping" is a strong one.
                 #This means the filtering process MUST achieve this.
                 #A more robust way:
                 # If curr_E <= last_L (overlap):
                 #    If curr_D < last_D:
                 #        final_merged_intervals[-1] = (last_E, curr_E - epsilon, last_D) // Truncate last
                 #        if final_merged_intervals[-1][0] > final_merged_intervals[-1][1]: final_merged_intervals.pop()
                 #        final_merged_intervals.append((curr_E, curr_L, curr_D)) // Add current
                 #    Else if curr_D == last_D: (handled by merge above)
                 #    Else (curr_D > last_D): // Current is worse over the overlap
                 #        temp_E = last_L + epsilon
                 #        if temp_E <= curr_L:
                 #            final_merged_intervals.append((temp_E, curr_L, curr_D)) // Add truncated current
                 # else: (no overlap)
                 #    final_merged_intervals.append((curr_E, curr_L, curr_D))

                # Returning just the Pareto set and relying on later dominance (Prop 3.1) to resolve.
                # For now, let's stick to the simpler merge for same-cost, and add if distinct non-overlapping.
                # The code above already did the simple merge. If it didn't merge and wasn't disjoint, it's an issue.
                # Let's return the sorted Pareto list as `dominant_intervals`. The non-overlapping property is hard to guarantee
                # perfectly without the exact procedure from Hoogeboom et al. [forthcoming].
                # The provided Lemma 2.1 proof uses Prop 2.1 (from Chapter 2, for duration) to argue that
                # if intervals are not properly ordered (L_q < E_{q+1}), one must dominate the other.
                # This means our Pareto filter (E1<=E2, L1>=L2, D1<=D2) is the key.
                pass # The dominant_intervals list is already sorted from Pareto step.

    # Return the result of Pareto dominance, sorted.
    return dominant_intervals


def extend_label_to_new_node(label_at_i: Label, next_node_id: int, data: dict) -> list:
    """
    Wrapper function for extending a label from node i to next_node_id (customer or depot).
    If next_node_id is a customer, uses _generate_candidate_intervals_at_j.
    If next_node_id is the depot, no new intervals are generated, but feasibility is checked.

    Args:
        label_at_i (Label): The label ending at node i.
        next_node_id (int): The ID of the next node to visit.
        data (dict): The preprocessed data dictionary.

    Returns:
        list: A list of new dominant forward start intervals for the path extended to next_node_id.
              Returns empty list if no feasible extension or if next_node_id is depot (as intervals
              are properties of customer service starts).
    """
    if next_node_id == data['depot_info']['id']: # Extending back to depot
        # No new "forward start intervals" are typically defined for the end depot.
        # Feasibility of reaching end depot is checked when forming a full route.
        # The "dominant_forward_start_intervals" attribute is for service start at a customer.
        # The cost to reach the end depot is simply D_i + travel_dist(i, depot_end).
        # The "time" for the label reaching end depot will be the path completion time.
        return [] # Or perhaps special handling if labels are to store completion times at depot.
                  # For now, consistent with paper, these intervals are for customer service.
    else: # Extending to another customer
        candidate_intervals = _generate_candidate_intervals_at_j(label_at_i, next_node_id, data)
        if not candidate_intervals:
            return []
        return _filter_dominant_intervals_pareto(candidate_intervals)

In [8]:
import math
# Assuming Label class, initialize_depot_label, extend_label_to_new_node,
# _calculate_tau_ij_from_paper are defined as in previous steps.

def _calculate_phi_L1_L2(label1: Label, label2: Label, data: dict) -> float:
    """
    Calculates phi(L_f^1, L_f^2) as per Algorithm 3.1 in the paper[cite: 345].
    This version adapts for a travel distance objective. The term
    max{x - L_L_f1(y), 0} in the paper's phi calculation (line 6 of Alg 3.1)
    relates to additional waiting time if L1 finishes service earlier at L_L_f1(y)
    but has to wait until x. For a pure travel distance objective, this waiting
    does not add to the distance cost. So, that term effectively becomes 0.

    Thus, phi becomes max(D_L1(y_for_x) - D_L2(z_for_x)), where D is accumulated distance.
    Algorithm 3.1 structure is to iterate over intervals of L2 (z) and L1 (y).
    """
    phi_val = -float('inf')
    epsilon = 1e-6 # A small value as used in paper's context for strict inequalities

    # dominant_forward_start_intervals are (E, L, D)
    intervals_L1 = label1.dominant_forward_start_intervals
    intervals_L2 = label2.dominant_forward_start_intervals

    if not intervals_L1 or not intervals_L2: # Should not happen if labels are valid
        return -float('inf')

    idx_y = 0
    for E_L2_z, L_L2_z, D_L2_z in intervals_L2:
        # Find the interval y in L1 that is active for departures x in [E_L2_z, L_L2_z]
        # This requires careful handling of how delta_L1(x) is determined.
        # Algorithm 3.1 compares an interval z of L2 with inter-interval gaps of L1
        # or overlaps with intervals of L1.

        # Iterate through L1's intervals/gaps relevant to L2's current interval z
        current_L1_idx = 0
        temp_max_for_z = -float('inf')

        # This loop implements the logic from Algorithm 3.1 lines 3-11 [cite: 345]
        # It iterates through y in F_L1 (intervals_L1)
        # For each z in F_L2 (current L2 interval)
        temp_y_idx = 0
        while temp_y_idx < len(intervals_L1):
            E_L1_y, L_L1_y, D_L1_y = intervals_L1[temp_y_idx]

            # Next interval start for L1, or infinity if y is the last
            E_L1_y_plus_1 = intervals_L1[temp_y_idx+1][0] if temp_y_idx + 1 < len(intervals_L1) else float('inf')

            # Check overlap: [E_L2_z, L_L2_z] with [E_L1_y, E_L1_y_plus_1[
            # This is the "departure time x" range considered in Alg 3.1
            overlap_start = max(E_L2_z, E_L1_y)
            overlap_end = min(L_L2_z, E_L1_y_plus_1 - epsilon)

            if overlap_start <= overlap_end: # If there's a relevant segment to check
                # x is chosen as the end of this overlapping segment (line 5 of Alg 3.1)
                # x = min(L_L2_z, E_L1_y_plus_1 - epsilon)
                # For this x, delta_L1(x) = D_L1_y + max(x - L_L1_y, 0)
                # For distance, max(x - L_L1_y, 0) is 0. So delta_L1(x) = D_L1_y
                # delta_L2(x) = D_L2_z (since x is within L2's interval z by definition of loop)
                current_phi_contribution = D_L1_y - D_L2_z
                if current_phi_contribution > temp_max_for_z:
                    temp_max_for_z = current_phi_contribution

            if L_L2_z < E_L1_y_plus_1 - epsilon : # Line 9 of Alg 3.1 condition
                break # Go to next start time interval of label L2 (outer loop)

            temp_y_idx += 1

        if temp_max_for_z > -float('inf'):
             if temp_max_for_z > phi_val:
                phi_val = temp_max_for_z
        # Handle case where L2 interval extends beyond all L1 intervals/gaps
        elif temp_y_idx == len(intervals_L1) and E_L2_z <= intervals_L1[-1][1]: # L2_z overlaps with last L1 interval
            E_L1_y_last, L_L1_y_last, D_L1_y_last = intervals_L1[-1]
            # Check overlap with [E_L1_y_last, L_L1_y_last]
            overlap_start = max(E_L2_z, E_L1_y_last)
            overlap_end = min(L_L2_z, L_L1_y_last)
            if overlap_start <= overlap_end:
                 current_phi_contribution = D_L1_y_last - D_L2_z
                 if current_phi_contribution > phi_val: # update overall phi_val directly
                    phi_val = current_phi_contribution


    return phi_val if phi_val > -float('inf') else 0.0 # If no valid comparison, phi might be 0 or some default


def check_dominance(label1: Label, label2: Label, data: dict) -> bool:
    """
    Checks if label1 dominates label2 based on Proposition 3.1[cite: 339, 88].
    Adapted for travel distance objective.

    Args:
        label1: The potentially dominating label.
        label2: The potentially dominated label.
        data: Preprocessed data.

    Returns:
        True if label1 dominates label2, False otherwise.
    """
    # Condition 1: v(L_f^1) = v(L_f^2)
    # This is implicitly true as this function will be called for labels ending at the same node.
    if label1.current_node_id != label2.current_node_id:
        return False # Should not happen if used correctly

    # Condition 2: S(L_f^1) subseteq S_bar(L_f^2) [cite: 339]
    # Using simplified S(L_f^1) subseteq S(L_f^2) for elementarity.
    # If L1 visits a customer not in L2, L1 cannot dominate L2 if L2 could still visit it.
    # For L1 to dominate L2, L1 must be "more general" or equally general.
    # This means L1 should not have visited *more* specific customers than L2.
    # So, label1.visited_customers_set must be a subset of or equal to label2.visited_customers_set
    if not label1.visited_customers_set.issubset(label2.visited_customers_set):
        return False

    # Condition 3: q(L_f^1) <= q(L_f^2) [cite: 339]
    if label1.current_load > label2.current_load:
        return False

    # Condition 4: E_L_f1(1) <= E_L_f2(1) [cite: 340]
    if label1.get_earliest_start_of_first_interval() > label2.get_earliest_start_of_first_interval():
        return False

    # Condition 5: phi(L_f^1, L_f^2) <= pi(L_f^1) - pi(L_f^2) [cite: 340]
    # where pi(L_f) is the sum of duals for label L_f.
    # The RHS is label1.accumulated_dual_value_sum - label2.accumulated_dual_value_sum
    # The paper's proof appendix C.2 uses c(L_f^1) - c(L_f^2) where c(L_f) is sum of duals[cite: 88].

    # If all previous conditions hold and L1 is strictly better in one resource
    # or has strictly lower dual sum for same resources, phi might not be needed.
    # But the paper's BCP includes it.

    # For distance, the accumulated_dual_value_sum is the "sum of duals".
    # The "cost" of a path in the objective is distance.
    # The condition 5 is about reduced cost comparison.
    # Reduced cost of L1 + extension <= Reduced cost of L2 + extension
    # (Dist1 + phi) - DualSum1 <= Dist2 - DualSum2 (if phi represents cost difference)
    # Dist1 - DualSum1 <= Dist2 - (DualSum2 + phi)
    # The paper's condition (Prop 3.1, page 47): phi(L1,L2) <= pi(L1) - pi(L2)
    # where pi(L) is sum of dual variables.

    # Calculate phi(label1, label2)
    # This phi should represent max(cost_L1_extension - cost_L2_extension) for any common extension.
    # If cost is just distance, and delta_Lf(x) = D_Lf(y), then phi simplifies.
    # Let's use the _calculate_phi_L1_L2 implementation.
    phi_1_2 = _calculate_phi_L1_L2(label1, label2, data)

    if phi_1_2 > (label1.accumulated_dual_value_sum - label2.accumulated_dual_value_sum) + 1e-6: # Add tolerance for float comparison
        return False

    # At least one of the conditions (load, earliest_start_time, or effective cost from Cond 5)
    # must be strictly better if the visited sets are identical. Or if L1 is a strict subset for visited.
    if label1.visited_customers_set == label2.visited_customers_set:
        if not (label1.current_load < label2.current_load - 1e-6 or
                label1.get_earliest_start_of_first_interval() < label2.get_earliest_start_of_first_interval() - 1e-6 or
                (phi_1_2 < (label1.accumulated_dual_value_sum - label2.accumulated_dual_value_sum) - 1e-6) ):
            # If all are equal or only phi makes them equal, it might not be a strict dominance
            # However, the paper does not require strict inequality for dominance.
            # Let's assume non-strict as per paper for now.
            pass


    return True


def solve_espprc_forward_labeling(
    data: dict,
    dual_prices: dict,
    alpha_penalty: float,
    depot_return_time_limit: float
):
    """
    Solves the Elementary Shortest Path Problem with Resource Constraints (ESPPRC)
    using a monodirectional forward labeling algorithm.

    Args:
        data (dict): Preprocessed problem data.
        dual_prices (dict): {customer_id: dual_value} from RMP.
        alpha_penalty (float): Cost for using a vehicle.
        depot_return_time_limit (float): Latest allowed arrival time back at the depot.


    Returns:
        list: A list of new routes (dicts) with negative reduced costs.
              Each route dict: {'path': list_of_nodes, 'cost': travel_distance,
                                'customers_served': list_of_customers,
                                'reduced_cost': float, 'id': str}
    """
    depot_id = data['depot_info']['id']
    customers_list = data['customers_list']
    vehicle_capacity = data['vehicle_capacity']

    initial_label = initialize_depot_label(data)

    # Stores non-dominated labels found so far for each node
    # key: node_id, value: list of Label objects
    non_dominated_labels_at_node = {node_id: [] for node_id in [depot_id] + customers_list}
    non_dominated_labels_at_node[depot_id].append(initial_label)

    # Labels to be processed
    unprocessed_labels = [initial_label]

    generated_negative_routes = []
    min_reduced_cost_overall = 0.0 # Track the most negative reduced cost

    iteration_count = 0 # Safety break for very long runs

    while unprocessed_labels:
        iteration_count += 1
        # if iteration_count % 100 == 0:
        #     print(f"  ESPPRC Iteration: {iteration_count}, Unprocessed: {len(unprocessed_labels)}, Neg Routes: {len(generated_negative_routes)}")
        # if iteration_count > 20000: # Safety break
        #     print("  ESPPRC safety break: Max iterations reached.")
        #     break

        current_L = unprocessed_labels.pop(0) # FIFO processing for now

        # Try to extend to other customers
        for next_cust_id in customers_list:
            if next_cust_id not in current_L.visited_customers_set:
                # 1. Check capacity
                if current_L.current_load + data['demands'].get(next_cust_id, 0) <= vehicle_capacity:
                    # 2. Generate dominant forward start intervals at next_cust_id
                    # extend_label_to_new_node was the previous name
                    new_intervals_at_next_node = extend_label_to_new_node(current_L, next_cust_id, data)

                    if new_intervals_at_next_node: # Feasible extension found
                        new_path_seq = current_L.path_sequence + [next_cust_id]
                        new_visited_set = current_L.visited_customers_set.union({next_cust_id})
                        new_load = current_L.current_load + data['demands'].get(next_cust_id, 0)
                        new_dual_sum = current_L.accumulated_dual_value_sum + dual_prices.get(next_cust_id, 0.0)

                        new_label = Label(
                            current_node_id=next_cust_id,
                            visited_customers_set=new_visited_set,
                            current_load=new_load,
                            dominant_forward_start_intervals=new_intervals_at_next_node,
                            accumulated_dual_value_sum=new_dual_sum,
                            path_sequence=new_path_seq
                        )

                        # Dominance Check (Proposition 3.1)
                        is_dominated_by_existing = False
                        kept_labels_for_next_node = []
                        for existing_label in non_dominated_labels_at_node[next_cust_id]:
                            if check_dominance(existing_label, new_label, data):
                                is_dominated_by_existing = True
                                break
                            if not check_dominance(new_label, existing_label, data):
                                kept_labels_for_next_node.append(existing_label)

                        if not is_dominated_by_existing:
                            non_dominated_labels_at_node[next_cust_id] = kept_labels_for_next_node + [new_label]
                            unprocessed_labels.append(new_label)
                            # Sort unprocessed_labels? e.g. by path length or dual sum for heuristic processing order
                            # unprocessed_labels.sort(key=lambda l: len(l.path_sequence))


        # Try to extend current_L (ending at current_L.current_node_id) back to the depot
        if current_L.current_node_id != depot_id : # Path has at least one customer
            path_ending_node_id = current_L.current_node_id

            # tau_ij for returning to depot: service_at_path_ending_node + travel_time_to_depot
            tau_to_depot = _calculate_tau_ij_from_paper(path_ending_node_id, depot_id, data)
            dist_to_depot = data['travel_distances'].get((path_ending_node_id, depot_id), float('inf'))

            min_route_distance_for_this_path = float('inf')
            feasible_return = False

            for E_L_y, L_L_y, D_L_y in current_L.dominant_forward_start_intervals:
                # Earliest completion time at path_ending_node_id if service started at E_L_y
                # Service starts at E_L_y, duration is service_time. Travel starts after service.
                # Arrival at depot: E_L_y (start service) + tau_to_depot
                arrival_at_depot = E_L_y + tau_to_depot

                if arrival_at_depot <= depot_return_time_limit:
                    current_total_dist = D_L_y + dist_to_depot
                    if current_total_dist < min_route_distance_for_this_path:
                        min_route_distance_for_this_path = current_total_dist
                    feasible_return = True

            if feasible_return:
                # Route cost = total_travel_distance + alpha_penalty
                route_cost_C_r = min_route_distance_for_this_path + alpha_penalty
                # Reduced cost = C_r - sum(dual_prices for customers in path)
                # current_L.accumulated_dual_value_sum already has sum for customers in path_sequence[1:]
                sum_duals_for_path = current_L.accumulated_dual_value_sum

                reduced_cost = route_cost_C_r - sum_duals_for_path

                if reduced_cost < 1e-6 : # Threshold for negativity
                    # print(f"  Found path {current_L.path_sequence + [depot_id]} to depot. Dist: {min_route_distance_for_this_path:.2f}, RC: {reduced_cost:.2f}")
                    route_details = {
                        'path': current_L.path_sequence + [depot_id],
                        'cost': min_route_distance_for_this_path, # This is pure travel distance
                        'customers_served': list(current_L.visited_customers_set),
                        'reduced_cost': reduced_cost,
                        'id': f"gen_route_{len(generated_negative_routes)}_{iteration_count}"
                    }
                    generated_negative_routes.append(route_details)
                if reduced_cost < min_reduced_cost_overall:
                    min_reduced_cost_overall = reduced_cost

    # Sort by reduced cost to return the best ones first (most negative)
    generated_negative_routes.sort(key=lambda r: r['reduced_cost'])
    return generated_negative_routes

In [9]:
from typing import Optional, List, Dict, Tuple, Set

def extend_backward_label_to_node(label_at_j: Label, prev_node_id: int, data: dict) -> Optional[Label]:
    """
    Extends a backward label from node j to previous node i.

    This is the backward equivalent of forward label extension.
    The key difference is that we're going backwards: from j to i.

    Returns:
        New backward label at node i, or None if extension is infeasible.
    """
    if not label_at_j.is_forward:  # Must be backward label
        node_j_id = label_at_j.current_node_id

        # Check if prev_node_id is reachable
        if prev_node_id in label_at_j.visited_customers_set:
            return None

        # Check capacity
        if prev_node_id != data['depot_id']:
            new_load = label_at_j.current_load + data['demands'].get(prev_node_id, 0)
            if new_load > data['vehicle_capacity']:
                return None
        else:
            new_load = label_at_j.current_load

        # Generate backward intervals at prev_node_id
        # Similar to forward but in reverse direction
        new_intervals = _generate_backward_intervals_at_i(label_at_j, prev_node_id, data)

        if not new_intervals:
            return None

        # Create new backward label
        new_visited = label_at_j.visited_customers_set.copy()
        if prev_node_id != data['depot_id']:
            new_visited.add(prev_node_id)

        new_path = [prev_node_id] + label_at_j.path_sequence

        new_dual_sum = label_at_j.accumulated_dual_value_sum
        if prev_node_id != data['depot_id']:
            new_dual_sum += data.get('dual_prices', {}).get(prev_node_id, 0)

        return Label(
            current_node_id=prev_node_id,
            visited_customers_set=new_visited,
            current_load=new_load,
            dominant_intervals=new_intervals,
            accumulated_dual_value_sum=new_dual_sum,
            path_sequence=new_path,
            is_forward=False
        )

    return None


def _generate_backward_intervals_at_i(label_at_j: Label, node_i_id: int, data: dict) -> list:
    """
    Generates backward start intervals at node i when extending backward from j.

    This is the backward equivalent of forward interval generation.
    Key: We need to ensure that if we start service at node i within these intervals,
    we can reach node j in time.
    """
    node_j_id = label_at_j.current_node_id
    generated_intervals = []

    # tau_ij includes service time at i and travel time from i to j
    service_time_at_i = data['service_times'].get(node_i_id, 0) if node_i_id != data['depot_id'] else 0
    travel_time_ij = data['travel_times'].get((node_i_id, node_j_id), float('inf'))
    tau_ij = service_time_at_i + travel_time_ij
    travel_dist_ij = data['travel_distances'].get((node_i_id, node_j_id), float('inf'))

    # Get time windows at node i
    time_windows_at_i = data['time_windows_data'].get(node_i_id, [])

    # For each backward interval at j, determine feasible intervals at i
    for E_Lb_y, L_Lb_y, D_Lb_y in label_at_j.dominant_intervals:
        for e_i_t, l_i_t in time_windows_at_i:
            # To start service at j at time E_Lb_y, we must finish at i by E_Lb_y - travel_time_ij
            # So we must start at i by E_Lb_y - tau_ij
            latest_start_at_i = E_Lb_y - tau_ij

            # We can start at i as early as e_i_t, but no later than min(l_i_t, latest_start_at_i)
            new_E_i = e_i_t
            new_L_i = min(l_i_t, latest_start_at_i)

            if new_E_i <= new_L_i:
                # This is a feasible backward interval
                new_D_i = D_Lb_y + travel_dist_ij
                generated_intervals.append((new_E_i, new_L_i, new_D_i))

    # Filter to keep only dominant intervals
    return _filter_dominant_intervals_pareto(generated_intervals)


def check_backward_dominance(label1: Label, label2: Label, data: dict) -> bool:
    """
    Checks if backward label1 dominates backward label2 (Proposition 3.2).

    Conditions:
    1. v(L1_b) = v(L2_b)
    2. S(L1_b) ⊆ S_bar(L2_b)
    3. q(L1_b) ≤ q(L2_b)
    4. L_L1_b(|B_L1_b|) ≥ L_L2_b(|B_L2_b|)
    5. φ(L1_b, L2_b) ≤ π(L1_b) - π(L2_b)
    """
    if label1.current_node_id != label2.current_node_id:
        return False

    # Check if L1 visits only customers that L2 could still visit
    if not label1.visited_customers_set.issubset(label2.visited_customers_set):
        return False

    # Check load
    if label1.current_load > label2.current_load:
        return False

    # Check latest start time of last interval
    if label1.get_latest_start_of_last_interval() < label2.get_latest_start_of_last_interval():
        return False

    # Calculate φ(L1_b, L2_b) - similar to forward but for backward
    phi_val = _calculate_phi_backward(label1, label2, data)

    if phi_val > (label1.accumulated_dual_value_sum - label2.accumulated_dual_value_sum) + 1e-6:
        return False

    return True


def _calculate_phi_backward(label1: Label, label2: Label, data: dict) -> float:
    """
    Calculates φ(L1_b, L2_b) for backward labels.
    Similar to forward phi calculation but adapted for backward direction.
    """
    phi_val = -float('inf')
    epsilon = 1e-6

    intervals_L1 = label1.dominant_intervals
    intervals_L2 = label2.dominant_intervals

    if not intervals_L1 or not intervals_L2:
        return -float('inf')

    # Similar logic to forward but adapted for backward
    for E_L2_z, L_L2_z, D_L2_z in intervals_L2:
        for idx, (E_L1_y, L_L1_y, D_L1_y) in enumerate(intervals_L1):
            # Check overlap and calculate phi contribution
            # This is simplified - full implementation would follow Algorithm 3.1 adapted for backward
            overlap_start = max(E_L2_z, E_L1_y)
            overlap_end = min(L_L2_z, L_L1_y)

            if overlap_start <= overlap_end:
                # For backward, the phi calculation is similar but considers arrival times
                current_phi = D_L2_z - D_L1_y
                if current_phi > phi_val:
                    phi_val = current_phi

    return phi_val if phi_val > -float('inf') else 0.0


def merge_forward_backward_labels(forward_label: Label, backward_label: Label, data: dict) -> Optional[dict]:
    """
    Merges a forward and backward label to create a complete route (Algorithm C.2).

    Returns:
        Dict with route information if merge is feasible, None otherwise.
    """
    # Check merge conditions
    if forward_label.current_node_id == backward_label.current_node_id:
        return None  # Can't merge at same node

    # Check if arc exists
    if (forward_label.current_node_id, backward_label.current_node_id) not in data['travel_times']:
        return None

    # Check no shared customers
    if forward_label.visited_customers_set.intersection(backward_label.visited_customers_set):
        return None

    # Check capacity
    if forward_label.current_load + backward_label.current_load > data['vehicle_capacity']:
        return None

    # Check time feasibility
    tau_ij = _calculate_tau_ij_from_paper(forward_label.current_node_id,
                                          backward_label.current_node_id, data)

    # Find feasible interval combinations
    min_route_distance = float('inf')
    feasible_merge = False

    for E_Lf_y, L_Lf_y, D_Lf_y in forward_label.dominant_intervals:
        for E_Lb_z, L_Lb_z, D_Lb_z in backward_label.dominant_intervals:
            # Check if we can depart from forward node and arrive at backward node in time
            earliest_arrival_at_backward = E_Lf_y + tau_ij
            latest_arrival_at_backward = L_Lf_y + tau_ij

            # We need to arrive before L_Lb_z to start service by then
            if earliest_arrival_at_backward <= L_Lb_z:
                # Calculate waiting time if any
                actual_start_at_backward = max(earliest_arrival_at_backward, E_Lb_z)
                waiting_time = max(0, E_Lb_z - latest_arrival_at_backward)

                # Total distance for this combination
                dist_ij = data['travel_distances'][(forward_label.current_node_id,
                                                   backward_label.current_node_id)]
                total_dist = D_Lf_y + dist_ij + D_Lb_z

                if total_dist < min_route_distance:
                    min_route_distance = total_dist
                    feasible_merge = True

    if not feasible_merge:
        return None

    # Create complete path
    complete_path = forward_label.path_sequence + backward_label.path_sequence

    # Calculate route details
    route_details = {
        'path': complete_path,
        'cost': min_route_distance,  # Pure travel distance
        'customers_served': list(forward_label.visited_customers_set.union(
                                backward_label.visited_customers_set)),
        'reduced_cost': None,  # Will be calculated later
        'id': f"route_{len(complete_path)}_{forward_label.current_node_id}_{backward_label.current_node_id}"
    }

    return route_details


def solve_espprc_bidirectional(data: dict,
                               dual_prices: dict,
                               alpha_penalty: float,
                               max_labels: int = 50000) -> List[dict]:
    """
    Solves ESPPRC using bidirectional labeling algorithm.

    Returns:
        List of routes with negative reduced cost.
    """
    depot_id = data['depot_id']
    customers_list = data['customers_list']

    # Initialize forward and backward labels at depot
    initial_forward = initialize_depot_label(data)
    initial_backward = initialize_depot_label(data)
    initial_backward.is_forward = False

    # Storage for non-dominated labels
    forward_labels = {node_id: [] for node_id in [depot_id] + customers_list}
    backward_labels = {node_id: [] for node_id in [depot_id] + customers_list}

    forward_labels[depot_id].append(initial_forward)
    backward_labels[depot_id].append(initial_backward)

    # Unprocessed labels
    forward_unprocessed = [initial_forward]
    backward_unprocessed = [initial_backward]

    # Track bounds for merging decision
    min_forward_E = 0  # Minimum E_Lf(1) among forward labels
    max_backward_L = data['depot_info']['time_window'][1]  # Maximum L_Lb(|B_Lb|)

    generated_routes = []
    total_labels_processed = 0

    while (forward_unprocessed or backward_unprocessed) and total_labels_processed < max_labels:
        # Process forward labels
        if forward_unprocessed and total_labels_processed < max_labels // 2:
            current_forward = forward_unprocessed.pop(0)
            total_labels_processed += 1

            # Extend to customers
            for next_cust_id in customers_list:
                if next_cust_id not in current_forward.visited_customers_set:
                    # Check basic feasibility
                    if (current_forward.current_load + data['demands'].get(next_cust_id, 0) <=
                        data['vehicle_capacity']):
                        # Generate new forward intervals
                        new_intervals = extend_label_to_new_node(current_forward, next_cust_id, data)

                        if new_intervals:
                            # Create new forward label
                            new_path = current_forward.path_sequence + [next_cust_id]
                            new_visited = current_forward.visited_customers_set.union({next_cust_id})
                            new_load = current_forward.current_load + data['demands'][next_cust_id]
                            new_dual_sum = current_forward.accumulated_dual_value_sum + dual_prices.get(next_cust_id, 0)

                            new_forward_label = Label(
                                current_node_id=next_cust_id,
                                visited_customers_set=new_visited,
                                current_load=new_load,
                                dominant_intervals=new_intervals,
                                accumulated_dual_value_sum=new_dual_sum,
                                path_sequence=new_path,
                                is_forward=True
                            )

                            # Check dominance
                            is_dominated = False
                            labels_to_remove = []

                            for existing_label in forward_labels[next_cust_id]:
                                if check_dominance(existing_label, new_forward_label, data):
                                    is_dominated = True
                                    break
                                elif check_dominance(new_forward_label, existing_label, data):
                                    labels_to_remove.append(existing_label)

                            if not is_dominated:
                                # Remove dominated labels
                                for label in labels_to_remove:
                                    forward_labels[next_cust_id].remove(label)
                                    if label in forward_unprocessed:
                                        forward_unprocessed.remove(label)

                                # Add new label
                                forward_labels[next_cust_id].append(new_forward_label)
                                forward_unprocessed.append(new_forward_label)

                                # Update min forward E
                                min_forward_E = min(min_forward_E, new_forward_label.get_earliest_start_of_first_interval())

        # Process backward labels
        if backward_unprocessed and total_labels_processed < max_labels:
            current_backward = backward_unprocessed.pop(0)
            total_labels_processed += 1

            # Extend to customers (backwards)
            for prev_cust_id in customers_list:
                if prev_cust_id not in current_backward.visited_customers_set:
                    # Use backward extension
                    new_backward_label = extend_backward_label_to_node(current_backward, prev_cust_id, data)

                    if new_backward_label:
                        # Check backward dominance
                        is_dominated = False
                        labels_to_remove = []

                        for existing_label in backward_labels[prev_cust_id]:
                            if check_backward_dominance(existing_label, new_backward_label, data):
                                is_dominated = True
                                break
                            elif check_backward_dominance(new_backward_label, existing_label, data):
                                labels_to_remove.append(existing_label)

                        if not is_dominated:
                            # Remove dominated labels
                            for label in labels_to_remove:
                                backward_labels[prev_cust_id].remove(label)
                                if label in backward_unprocessed:
                                    backward_unprocessed.remove(label)

                            # Add new label
                            backward_labels[prev_cust_id].append(new_backward_label)
                            backward_unprocessed.append(new_backward_label)

                            # Update max backward L
                            max_backward_L = max(max_backward_L,
                                               new_backward_label.get_latest_start_of_last_interval())

        # Check merging condition
        if min_forward_E > max_backward_L or total_labels_processed >= max_labels // 2:
            # Merge forward and backward labels
            for forward_node_id, forward_label_list in forward_labels.items():
                if forward_node_id == depot_id:
                    continue

                for forward_label in forward_label_list:
                    # Try to merge with backward labels at different nodes
                    for backward_node_id, backward_label_list in backward_labels.items():
                        if backward_node_id == depot_id or backward_node_id == forward_node_id:
                            continue

                        for backward_label in backward_label_list:
                            # Try to merge
                            route_details = merge_forward_backward_labels(forward_label,
                                                                        backward_label, data)

                            if route_details:
                                # Calculate reduced cost
                                route_cost = route_details['cost'] + alpha_penalty
                                sum_duals = (forward_label.accumulated_dual_value_sum +
                                           backward_label.accumulated_dual_value_sum)
                                reduced_cost = route_cost - sum_duals

                                route_details['reduced_cost'] = reduced_cost

                                if reduced_cost < -1e-6:
                                    generated_routes.append(route_details)

            # Clear processed labels to make room for new ones
            if total_labels_processed >= max_labels:
                break

    # Sort routes by reduced cost
    generated_routes.sort(key=lambda r: r['reduced_cost'])

    return generated_routes

In [10]:
def solve_espprc_with_multiple_time_windows(
    customers_list, demands, service_times, time_windows_data,
    coordinates_data, vehicle_capacity, speed, alpha_penalty,
    dual_prices, depot_id=0, depot_return_time_limit=float('inf')):
    """
    Modified to use forward labeling for testing.
    """
    data = {
        'customers_list': customers_list,
        'demands': demands,
        'service_times': service_times,
        'time_windows_data': time_windows_data,
        'coordinates_data': coordinates_data,
        'vehicle_capacity': vehicle_capacity,
        'speed': speed,
        'depot_info': {
            'id': depot_id,
            'time_window': (0, depot_return_time_limit), # Use depot_return_time_limit
            'demand': 0,
            'service_time': 0
        },
        'dual_prices': dual_prices,
        'travel_times': {},
        'travel_distances': {}
    }

    # Populate travel times and distances
    data['travel_distances'] = calculate_distance_matrix(coordinates_data)
    data['travel_times'] = calculate_travel_time_matrix(data['travel_distances'], speed)

    # Call forward labeling ESPPRC
    routes = solve_espprc_forward_labeling(data, dual_prices, alpha_penalty, depot_return_time_limit)

    if routes:
        best_route = routes[0] # Already sorted by reduced cost
        return best_route, best_route['reduced_cost']
    else:
        return None, float('inf')

# --- Utility functions for the test instance ---
def calculate_distance_matrix(coordinates: Dict[int, Tuple[float, float]]) -> Dict[Tuple[int, int], float]:
    distances = {}
    ids = list(coordinates.keys())
    for i in range(len(ids)):
        for j in range(len(ids)):
            id1 = ids[i]
            id2 = ids[j]
            if id1 == id2:
                distances[(id1, id2)] = 0.0
            else:
                coord1 = coordinates[id1]
                coord2 = coordinates[id2]
                dist = math.sqrt((coord1[0] - coord2[0])**2 + (coord1[1] - coord2[1])**2)
                distances[(id1, id2)] = dist
    return distances

def calculate_travel_time_matrix(distances: Dict[Tuple[int, int], float], speed: float) -> Dict[Tuple[int, int], float]:
    travel_times = {}
    for pair, dist in distances.items():
        travel_times[pair] = dist / speed
    return travel_times

In [11]:
# Test Instance Data
customer_ids = [1, 2]
depot_id = 0

coordinates = {
    depot_id: (0, 0),
    1: (10, 0),
    2: (0, 20),
}

demands_kg = {
    depot_id: 0,
    1: 50,
    2: 70,
}

# Service times in minutes
service_times_min = {
    depot_id: 0,
    1: 10,
    2: 15,
}

# Time windows: list of (start_min, end_min) from midnight
time_windows_min = {
    depot_id: [(0, 1000)], # Depot operational window
    1: [(60, 120), (180, 240)],
    2: [(90, 150)],
}

vehicle_capacity_kg = 150.0
truck_speed = 1.0 # distance units per minute
alpha_penalty_dist_equivalent = 25.0
max_cg_iter = 3
depot_latest_return_time = 1000 # Matches depot window end

# --- Calculate distances and travel times for the instance ---
dist_matrix = calculate_distance_matrix(coordinates)
time_matrix = calculate_travel_time_matrix(dist_matrix, truck_speed)

# --- Prepare initial routes ---
# For simplicity, let's create very basic initial routes.
# Route 1: Depot -> Cust 1 -> Depot
# Route 2: Depot -> Cust 2 -> Depot
# We need their *travel distance* for the 'cost' field.
# Feasibility of these initial routes regarding time windows is important.
# We can use your evaluate_route function if it's robust enough for simple paths,
# or manually find a feasible cost.

# Prepare data for evaluate_route
eval_data = {
    'customers_list': customer_ids,
    'demands': demands_kg,
    'service_times': service_times_min,
    'time_windows_data': time_windows_min,
    'coordinates_data': coordinates,
    'vehicle_capacity': vehicle_capacity_kg,
    'speed': truck_speed,
    'depot_id': depot_id,
    'depot_info': {
        'id': depot_id,
        'time_window': time_windows_min[depot_id][0],
        'demand': 0,
        'service_time': 0
    },
    'travel_times': time_matrix,
    'travel_distances': dist_matrix
}

# Try to find feasible initial routes (requires careful departure time selection)
# For this test, let's assume some feasible costs if evaluate_route is too complex to debug now
# Cost for D-1-D: dist(0,1)+dist(1,0) = 10+10=20.
#   Depot (0) -> Cust 1 (10,0) -> Depot (0,0).
#   Depart depot at 0. Arrive C1 at 0+dist(0,1)=10. C1 TWs: [(60,120), (180,240)]. Must wait.
#   If service starts at 60. Finishes at 60+10(service)=70.
#   Depart C1 at 70. Arrive depot at 70+dist(1,0)=70+10=80. Feasible.
cost1_eval = evaluate_route(route_sequence=[1], departure_time=0, data=eval_data)
if cost1_eval is None: # Try a later departure for C1's second window
    cost1_eval = evaluate_route(route_sequence=[1], departure_time=180-service_times_min[1]-time_matrix[(0,1)]-1, data=eval_data) # Try to arrive just before 180
if cost1_eval is None:
    print("Warning: Could not find feasible initial route for Cust 1 automatically. Using raw distance.")
    cost1_eval = dist_matrix[(0,1)] + dist_matrix[(1,0)]


# Cost for D-2-D: dist(0,2)+dist(2,0) = 20+20=40
#   Depot (0) -> Cust 2 (0,20) -> Depot (0,0)
#   Depart depot at 0. Arrive C2 at 0+dist(0,2)=20. C2 TW: [(90,150)]. Must wait.
#   Service starts at 90. Finishes at 90+15(service)=105.
#   Depart C2 at 105. Arrive depot at 105+dist(2,0)=105+20=125. Feasible.
cost2_eval = evaluate_route(route_sequence=[2], departure_time=0, data=eval_data)
if cost2_eval is None:
    print("Warning: Could not find feasible initial route for Cust 2 automatically. Using raw distance.")
    cost2_eval = dist_matrix[(0,2)] + dist_matrix[(2,0)]

initial_routes_for_solver = [
    {'id': 'initial_A', 'cost': cost1_eval if cost1_eval is not None else 20.0, 'customers_served': [1]},
    {'id': 'initial_B', 'cost': cost2_eval if cost2_eval is not None else 40.0, 'customers_served': [2]},
]
print(f"Initial Route A cost (Cust 1): {initial_routes_for_solver[0]['cost']}")
print(f"Initial Route B cost (Cust 2): {initial_routes_for_solver[1]['cost']}")

Initial Route A cost (Cust 1): 20.0
Initial Route B cost (Cust 2): 40.0


In [12]:
import math
import gurobipy as gp
from gurobipy import GRB
from typing import List, Dict, Tuple, Optional, Set # Added Set for type hinting

# --- Main test execution ---
if __name__ == "__main__":
    print("Starting Column Generation Test...")

    # Ensure all data uses the correct IDs (depot_id for depot, customer_ids for customers)
    # The 'customers_list' for RMP and ESPPRC should only contain actual customer IDs, not the depot.

    final_model, selected_routes, all_generated_routes = column_generation_solver(
        initial_routes=initial_routes_for_solver,
        customers_list=customer_ids, # Pass only customer IDs
        demands=demands_kg,
        service_times=service_times_min,
        time_windows_data=time_windows_min,
        coordinates_data=coordinates,
        vehicle_capacity=vehicle_capacity_kg,
        speed=truck_speed,
        alpha_penalty=alpha_penalty_dist_equivalent,
        max_cg_iterations=max_cg_iter
    )

    if selected_routes:
        print("\n--- Test Run Completed Successfully ---")
        print(f"Number of selected routes: {len(selected_routes)}")
        total_dist = sum(r['cost'] for r in selected_routes)
        print(f"Total travel distance of selected routes: {total_dist:.2f}")
        obj_val_check = total_dist + alpha_penalty_dist_equivalent * len(selected_routes)
        print(f"Calculated Objective (dist + alpha*veh): {obj_val_check:.2f}")
        if final_model:
             print(f"Gurobi Master IP Objective: {final_model.ObjVal:.2f}")

        for i, r_detail in enumerate(selected_routes):
             print(f"  Selected Route {i+1} (ID {r_detail.get('id', 'N/A')}):")
             print(f"    Customers: {r_detail['customers_served']}")
             print(f"    Travel Distance (cost): {r_detail['cost']:.2f}")
             # You might want to re-evaluate the final selected routes with evaluate_route
             # to verify their true feasibility and timing details.
    else:
        print("\n--- Test Run Did Not Produce a Final Solution ---")

    print(f"\nTotal routes in pool at end: {len(all_generated_routes)}")

Starting Column Generation Test...

--- Column Generation Iteration: 1 ---
Current number of routes in RMP: 2
Restricted license - for non-production use only - expires 2026-11-23
RMP Objective Value: 110.00
Found new route with reduced cost: -32.64

--- Column Generation Iteration: 2 ---
Current number of routes in RMP: 3
RMP Objective Value: 77.36
No new route with negative reduced cost found. LP optimal for current columns.

--- Column Generation Finished ---
Final number of routes generated: 3

Solving final Master Problem as IP...
Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2652745
Set parameter Threads to value 95
Academic license 2652745 - for non-commercial use only - registered to di___@student.sdu.dk
Master IP Objective: 77.36067977499789
Number of vehicles/routes used: 1

Final Solution from Master IP:
  Objective (IP): 77.36 (This includes alpha in C_r)
  Calculated objective (dist + alpha*veh): 77.36
  Number of vehicles: 1
  Total tr

In [17]:
# Step 1: Make sure all your Python functions (Label class, solvers, data loaders, etc.)
# have been defined and executed in previous cells in this Colab notebook.

# Step 2: Upload your instance files
from google.colab import files
import os

print("Please upload your instance files (e.g., instance_coordinates.txt, instance_demand.txt, etc.)")
print("Ensure they match the names expected by your 'load_and_preprocess_data' function (e.g., 'instance_*.txt')")

# Clean up any old files if they exist to avoid confusion
file_prefixes_to_remove = ["instance_coordinates.txt", "instance_demand.txt", "instance_service_time.txt", "instance_time_windows.txt"]
for f_name in file_prefixes_to_remove:
    if os.path.exists(f"/content/{f_name}"):
        os.remove(f"/content/{f_name}")
        print(f"Removed old /content/{f_name}")

uploaded = files.upload()

for fn in uploaded.keys():
  print(f'User uploaded file "{fn}" with length {len(uploaded[fn])} bytes')

# Verify that the expected files were uploaded (adjust instance_name if different)
INSTANCE_NAME_PREFIX = "instance" # This should match the prefix of your uploaded files
required_files = [
    f"{INSTANCE_NAME_PREFIX}_coordinates.txt",
    f"{INSTANCE_NAME_PREFIX}_demand.txt",
    f"{INSTANCE_NAME_PREFIX}_service_time.txt",
    f"{INSTANCE_NAME_PREFIX}_time_windows.txt"
]
all_files_present = True
for req_file in required_files:
    if not os.path.exists(f"/content/{req_file}"):
        print(f"ERROR: Required file /content/{req_file} was not uploaded or has a different name.")
        all_files_present = False

if not all_files_present:
    print("Please re-run the cell and ensure all required files with correct names are uploaded.")
else:
    print("\nAll required files seem to be uploaded. Proceeding with the test run...")

    # Step 3: Define parameters and run the test
    # (Assuming all necessary functions like load_and_preprocess_data, evaluate_route,
    # column_generation_solver, etc., are already defined in your notebook)

    if __name__ == "__main__": # Standard practice, though in Colab cells run sequentially
        print("\nStarting Column Generation Test (4 Customers with Uploaded Files)...")

        # Parameters for this test run
        test_depot_id = 0
        test_vehicle_capacity = 250.0
        test_speed_dist_per_hr = 60.0 # Results in 1.0 dist_unit/min for travel_time_matrix if dist/hr is expected
        test_alpha_penalty = 50.0
        test_max_cg_iter = 10
        test_depot_latest_return = 24 * 60 # minutes

        # Load data using your functions
        # The base_path will be "/content/" where Colab uploads files by default
        data_4_cust = load_and_preprocess_data(
            base_path="/content",
            instance_name=INSTANCE_NAME_PREFIX, # Use the prefix you defined
            speed=test_speed_dist_per_hr,
            vehicle_capacity=test_vehicle_capacity,
            depot_id=test_depot_id
        )

        # Prepare initial routes (single customer routes)
        initial_routes_for_solver = []
        if data_4_cust and data_4_cust.get('customers_list'): # Check if data loaded correctly
            for cust_id in data_4_cust['customers_list']:
                feasible_cost = None
                # Try departing early
                cost = evaluate_route(route_sequence=[cust_id], departure_time=0, data=data_4_cust)
                if cost is not None:
                    feasible_cost = cost
                else: # Try departing just in time for the first window
                    cust_tws = data_4_cust['time_windows_data'].get(cust_id, [])
                    if cust_tws:
                        first_tw_start = cust_tws[0][0]
                        travel_to_cust = data_4_cust['travel_times'].get((test_depot_id, cust_id), 0)
                        depart_depot_target = first_tw_start - travel_to_cust
                        cost = evaluate_route(route_sequence=[cust_id], departure_time=max(0, depart_depot_target), data=data_4_cust)
                        if cost is not None:
                            feasible_cost = cost

                if feasible_cost is not None:
                    initial_routes_for_solver.append({
                        'id': f'initial_cust_{cust_id}',
                        'cost': feasible_cost,
                        'customers_served': [cust_id]
                    })
                    print(f"Initial route for customer {cust_id} cost: {feasible_cost:.2f}")
                else:
                    dist_to = data_4_cust['travel_distances'].get((test_depot_id, cust_id), 0)
                    dist_from = data_4_cust['travel_distances'].get((cust_id, test_depot_id), 0)
                    raw_cost = dist_to + dist_from
                    initial_routes_for_solver.append({
                        'id': f'initial_cust_{cust_id}_raw',
                        'cost': raw_cost,
                        'customers_served': [cust_id]
                    })
                    print(f"Warning: Could not find feasible initial route for Cust {cust_id} via evaluate_route. Using raw distance: {raw_cost:.2f}")

            if not initial_routes_for_solver and data_4_cust['customers_list']:
                 print("Error: No initial routes could be created, but customers exist. Aborting CG.")
            else:
                # Run the column generation solver
                final_model, selected_routes, all_generated_routes = column_generation_solver(
                    initial_routes=initial_routes_for_solver,
                    customers_list=data_4_cust['customers_list'],
                    demands=data_4_cust['demands'],
                    service_times=data_4_cust['service_times'],
                    time_windows_data=data_4_cust['time_windows_data'],
                    coordinates_data=data_4_cust['coordinates_data'],
                    vehicle_capacity=test_vehicle_capacity,
                    speed=test_speed_dist_per_hr,
                    alpha_penalty=test_alpha_penalty,
                    max_cg_iterations=test_max_cg_iter
                )

                if selected_routes:
                    print("\n--- Test Run (4 Customers with Uploaded Files) Completed Successfully ---")
                    print(f"Number of selected routes: {len(selected_routes)}")
                    total_dist = sum(r['cost'] for r in selected_routes)
                    print(f"Total travel distance of selected routes: {total_dist:.2f}")

                    if final_model and hasattr(final_model, 'ObjVal') and final_model.SolCount > 0:
                         obj_val_check = total_dist + test_alpha_penalty * len(selected_routes)
                         print(f"Calculated Objective (dist + alpha*veh): {obj_val_check:.2f}")
                         print(f"Gurobi Master IP Objective: {final_model.ObjVal:.2f}")

                    for i, r_detail in enumerate(selected_routes):
                         print(f"  Selected Route {i+1} (ID {r_detail.get('id', 'N/A')}):")
                         print(f"    Customers: {r_detail['customers_served']}")
                         print(f"    Travel Distance (cost): {r_detail['cost']:.2f}")
                elif final_model: # Model exists but no selected routes
                    print("\n--- Test Run (4 Customers with Uploaded Files) Did Not Produce a Feasible Solution via Master IP ---")
                    print(f"Final IP Model status: {final_model.status}")
                else: # No model and no routes
                    print("\n--- Test Run (4 Customers with Uploaded Files) Did Not Produce a Final Solution (CG failed) ---")


                if all_generated_routes:
                    print(f"\nTotal routes in pool at end: {len(all_generated_routes)}")
                else:
                    print("\nNo routes in pool at end (or CG did not complete to that stage).")
        else:
            print("Error: Data loading failed or no customers found. Cannot proceed.")

Please upload your instance files (e.g., instance_coordinates.txt, instance_demand.txt, etc.)
Ensure they match the names expected by your 'load_and_preprocess_data' function (e.g., 'instance_*.txt')
Removed old /content/instance_coordinates.txt
Removed old /content/instance_demand.txt
Removed old /content/instance_service_time.txt
Removed old /content/instance_time_windows.txt


Saving instance_coordinates.txt to instance_coordinates.txt
Saving instance_demand.txt to instance_demand.txt
Saving instance_service_time.txt to instance_service_time.txt
Saving instance_time_windows.txt to instance_time_windows.txt
User uploaded file "instance_coordinates.txt" with length 3107 bytes
User uploaded file "instance_demand.txt" with length 1697 bytes
User uploaded file "instance_service_time.txt" with length 1293 bytes
User uploaded file "instance_time_windows.txt" with length 3833 bytes

All required files seem to be uploaded. Proceeding with the test run...

Starting Column Generation Test (4 Customers with Uploaded Files)...
Loading instance data...
Calculating distance and travel time matrices...
Loaded 201 customers
Depot: 0
Vehicle capacity: 250.0 kg
Speed: 60.0 distance units/hour


KeyError: (0, 1)

In [ ]:
# Ensure Gurobi is installed in your Colab environment:
# !pip install gurobipy

# Import necessary libraries (ensure these are also imported where your functions are defined)
import math
from typing import List, Dict, Tuple, Optional, Set, Any # Make sure these are imported in your function definitions too
import gurobipy as gp
from gurobipy import GRB
from google.colab import files
import os

# --- SAFETY CHECK: Ensure all required functions are defined ---
# It's good practice to quickly check if the main functions exist before proceeding.
# This won't check their internal correctness, just their presence.
required_functions = [
    "load_and_preprocess_data", "evaluate_route", "column_generation_solver",
    "Label", "initialize_depot_label", "_calculate_tau_ij_from_paper",
    "_generate_candidate_intervals_at_j", "_filter_dominant_intervals_pareto",
    "extend_label_to_new_node", "_calculate_phi_L1_L2", "check_dominance",
    "solve_espprc_forward_labeling", "solve_restricted_master_problem",
    "solve_master_ip", "solve_espprc_with_multiple_time_windows"
    # Add any other crucial functions your solver depends on
]
for func_name in required_functions:
    if func_name not in globals():
        print(f"ERROR: Required function '{func_name}' is not defined in the notebook.")
        print("Please make sure all your function definition cells have been executed.")
        # You might want to raise an error here to stop execution
        raise NameError(f"Function {func_name} not defined.")
print("All prerequisite functions seem to be defined.")

# Step 1: File Upload Section
print("\n--- File Upload for Actual Run ---")
print("Please upload your actual instance files.")
print("Ensure they are named according to the prefix expected by 'load_and_preprocess_data'.")

# Define the expected file name prefix for your actual instance
# For example, if your files are "actual_coords.txt", "actual_demand.txt",
# then INSTANCE_NAME_PREFIX_ACTUAL_RUN would be "actual"
INSTANCE_NAME_PREFIX_ACTUAL_RUN = input("Enter the common prefix for your instance files (e.g., 'instance', 'problemX'): ")

# Clean up any old files if they exist in /content/ that match the new prefix
# (Be careful with this if you have other important files with the same prefix)
files_to_potentially_remove = [
    f"{INSTANCE_NAME_PREFIX_ACTUAL_RUN}_coordinates.txt",
    f"{INSTANCE_NAME_PREFIX_ACTUAL_RUN}_demand.txt",
    f"{INSTANCE_NAME_PREFIX_ACTUAL_RUN}_service_time.txt",
    f"{INSTANCE_NAME_PREFIX_ACTUAL_RUN}_time_windows.txt"
]
for f_name in files_to_potentially_remove:
    if os.path.exists(f"/content/{f_name}"):
        print(f"Found existing file: /content/{f_name}. Removing it before upload...")
        os.remove(f"/content/{f_name}")

uploaded_actual = files.upload()

for fn_actual in uploaded_actual.keys():
  print(f'User uploaded file "{fn_actual}" with length {len(uploaded_actual[fn_actual])} bytes')

# Verify uploaded files
all_files_present_actual = True
required_files_actual = [
    f"{INSTANCE_NAME_PREFIX_ACTUAL_RUN}_coordinates.txt",
    f"{INSTANCE_NAME_PREFIX_ACTUAL_RUN}_demand.txt",
    f"{INSTANCE_NAME_PREFIX_ACTUAL_RUN}_service_time.txt",
    f"{INSTANCE_NAME_PREFIX_ACTUAL_RUN}_time_windows.txt"
]
for req_file_actual in required_files_actual:
    if not os.path.exists(f"/content/{req_file_actual}"):
        print(f"ERROR: Required file /content/{req_file_actual} was not uploaded or has a different name.")
        all_files_present_actual = False

if not all_files_present_actual:
    print("Please re-run the cell and ensure all required files for the actual instance are uploaded with correct names.")
else:
    print("\nAll required files for the actual run seem to be uploaded.")
    print("Proceeding with the solver execution...")

    # Step 2: Define Parameters for the Actual Run
    # These might be different from your test runs.
    # Your problem description mentions:
    # - 200 retailers (customers) - your instance files will reflect this
    # - Vehicle capacity Q = 12600 kg
    # - Speed of trucks = 1000 distance units / h

    actual_depot_id = int(input(f"Enter the DEPOT ID for instance '{INSTANCE_NAME_PREFIX_ACTUAL_RUN}' (e.g., 0 or 1): "))
    actual_vehicle_capacity = float(input(f"Enter the VEHICLE CAPACITY for instance '{INSTANCE_NAME_PREFIX_ACTUAL_RUN}' (e.g., 12600): ") or 12600.0)
    actual_speed_dist_per_hr = float(input(f"Enter the vehicle SPEED (distance units per HOUR) for instance '{INSTANCE_NAME_PREFIX_ACTUAL_RUN}' (e.g., 1000): ") or 1000.0)
    actual_alpha_penalty = float(input(f"Enter the ALPHA PENALTY (cost per vehicle) for instance '{INSTANCE_NAME_PREFIX_ACTUAL_RUN}' (e.g., 1000, 5000): ") or 1000.0) # Adjust as needed
    actual_max_cg_iter = int(input(f"Enter the MAXIMUM Column Generation ITERATIONS for instance '{INSTANCE_NAME_PREFIX_ACTUAL_RUN}' (e.g., 50, 100): ") or 50)
    actual_depot_planning_end_time_min = float(input(f"Enter the DEPOT latest return time (minutes from midnight, e.g., 1440 for 24h): ") or 24*60)

    # Step 3: Load and Preprocess Data for the Actual Run
    print(f"\nLoading and preprocessing data for instance: {INSTANCE_NAME_PREFIX_ACTUAL_RUN}...")
    actual_instance_data = None
    try:
        actual_instance_data = load_and_preprocess_data(
            base_path="/content",
            instance_name=INSTANCE_NAME_PREFIX_ACTUAL_RUN,
            speed=actual_speed_dist_per_hr,
            vehicle_capacity=actual_vehicle_capacity,
            depot_id=actual_depot_id
        )
    except Exception as e:
        print(f"Error during data loading and preprocessing: {e}")
        print("Please check your uploaded files and their format.")

    if actual_instance_data and actual_instance_data.get('customers_list'):
        print(f"\nData loaded successfully. Number of customers: {len(actual_instance_data['customers_list'])}")

        # Step 4: Prepare Initial Routes for the Actual Run
        # For a large instance (200 customers), creating good initial routes can be complex.
        # Starting with single-customer routes is a safe but potentially slow start for CG.
        # You might have a separate heuristic to generate better starting routes.
        # For now, let's stick to single-customer routes, but be aware this might be insufficient for very large problems.
        print("\nPreparing initial routes (single customer per route)...")
        initial_routes_actual_run = []
        for cust_id_actual in actual_instance_data['customers_list']:
            feasible_cost_actual = None
            # Try a few departure times
            departure_attempts = [0] # Start with departing at time 0
            cust_tws_actual = actual_instance_data['time_windows_data'].get(cust_id_actual, [])
            if cust_tws_actual: # Try to depart just in time for each window
                travel_to_cust_actual = actual_instance_data['travel_times'].get((actual_depot_id, cust_id_actual), 0)
                for tw_start, _ in cust_tws_actual:
                    departure_attempts.append(max(0, tw_start - travel_to_cust_actual))

            for dep_time in sorted(list(set(departure_attempts))): # Unique sorted departure attempts
                cost_actual = evaluate_route(route_sequence=[cust_id_actual], departure_time=dep_time, data=actual_instance_data)
                if cost_actual is not None:
                    if feasible_cost_actual is None or cost_actual < feasible_cost_actual:
                        feasible_cost_actual = cost_actual

            if feasible_cost_actual is not None:
                initial_routes_actual_run.append({
                    'id': f'initial_actual_cust_{cust_id_actual}',
                    'cost': feasible_cost_actual,
                    'customers_served': [cust_id_actual]
                })
                # print(f"Initial route for actual customer {cust_id_actual} cost: {feasible_cost_actual:.2f}")
            else:
                # Fallback if evaluate_route fails for all attempts
                dist_to_actual = actual_instance_data['travel_distances'].get((actual_depot_id, cust_id_actual), float('inf'))
                dist_from_actual = actual_instance_data['travel_distances'].get((cust_id_actual, actual_depot_id), float('inf'))
                if dist_to_actual != float('inf') and dist_from_actual != float('inf'):
                    raw_cost_actual = dist_to_actual + dist_from_actual
                    initial_routes_actual_run.append({
                        'id': f'initial_actual_cust_{cust_id_actual}_raw',
                        'cost': raw_cost_actual,
                        'customers_served': [cust_id_actual]
                    })
                    print(f"Warning: Could not find time-feasible initial route for Cust {cust_id_actual}. Using raw distance: {raw_cost_actual:.2f}")
                else:
                     print(f"ERROR: Cannot even calculate raw distance for initial route for Cust {cust_id_actual}.")


        if not initial_routes_actual_run and actual_instance_data['customers_list']:
            print("Error: No initial routes could be created for the actual run, but customers exist. Aborting solver.")
        elif not actual_instance_data['customers_list']:
             print("No customers loaded. Nothing to solve.")
        else:
            print(f"\nSuccessfully created {len(initial_routes_actual_run)} initial routes.")
            print("Starting the Column Generation Solver for the actual run...")

            # Step 5: Run the Column Generation Solver
            # ENSURE column_generation_solver DEFINITION ACCEPTS depot_id and depot_max_time
            final_model_actual, selected_routes_actual, all_generated_routes_actual = column_generation_solver(
                initial_routes=initial_routes_actual_run,
                customers_list=actual_instance_data['customers_list'], # Corrected: no _cg
                demands=actual_instance_data['demands'],                 # Corrected
                service_times=actual_instance_data['service_times'],     # Corrected
                time_windows_data=actual_instance_data['time_windows_data'], # Corrected
                coordinates_data=actual_instance_data['coordinates_data'], # Corrected
                vehicle_capacity=actual_vehicle_capacity,               # Corrected
                speed=actual_speed_dist_per_hr,                         # Corrected
                alpha_penalty=actual_alpha_penalty,                     # Corrected
                max_cg_iterations=actual_max_cg_iter,
                depot_id_cg=actual_depot_id,                               # Pass explicitly
                depot_max_time_cg=actual_depot_planning_end_time_min       # Pass explicitly
            )

            # Step 6: Report Results
            if selected_routes_actual:
                print("\n--- Actual Run Completed Successfully ---")
                num_vehicles_actual = len(selected_routes_actual)
                total_dist_actual = sum(r['cost'] for r in selected_routes_actual)
                print(f"Number of vehicles used: {num_vehicles_actual}")
                print(f"Total travel distance: {total_dist_actual:.2f}")

                if final_model_actual and hasattr(final_model_actual, 'ObjVal') and final_model_actual.SolCount > 0:
                     calculated_obj_actual = total_dist_actual + actual_alpha_penalty * num_vehicles_actual
                     print(f"Calculated Objective (dist + alpha*num_veh): {calculated_obj_actual:.2f}")
                     print(f"Gurobi Master IP Objective: {final_model_actual.ObjVal:.2f}")
                     # Optimality Gap (if RMP provides a valid lower bound from the last iteration)
                     # This requires capturing the final RMP objective before the IP solve.
                     # The current CG function doesn't return it, but it's printed.
                     # For a true gap, you'd need the final LP relaxation value of the full master problem.
                     # If final_ip_model.ObjBound is available and meaningful:
                     if hasattr(final_model_actual, 'ObjBound') and final_model_actual.ObjVal > 0: # Ensure ObjVal is positive for meaningful gap
                         gap = ( (final_model_actual.ObjVal - final_model_actual.ObjBound) / final_model_actual.ObjVal ) * 100
                         print(f"Optimality Gap (from Gurobi IP solve): {gap:.2f}% (ObjBound: {final_model_actual.ObjBound:.2f})")
                     else:
                         print("Optimality Gap information not readily available from Gurobi IP solve for this run.")


                print("\nSelected Routes Details:")
                for i, r_detail_actual in enumerate(selected_routes_actual):
                     print(f"  Route {i+1} (ID {r_detail_actual.get('id', 'N/A')}):")
                     print(f"    Customers: {r_detail_actual['customers_served']}")
                     print(f"    Travel Distance (cost): {r_detail_actual['cost']:.2f}")
                     # You might want to re-evaluate each selected route here with evaluate_route
                     # to confirm its full schedule and feasibility.
            elif final_model_actual:
                 print("\n--- Actual Run Did Not Produce a Feasible Solution via Master IP ---")
                 print(f"Final IP Model status: {final_model_actual.status}")
            else:
                print("\n--- Actual Run Did Not Produce a Final Solution (CG or IP failed more fundamentally) ---")

            if all_generated_routes_actual:
                print(f"\nTotal routes in pool at end of CG: {len(all_generated_routes_actual)}")
            else:
                print("\nNo routes in pool at end (or CG did not complete to that stage).")
    else:
        print("\n--- Actual Run Aborted: Data loading failed or no customers found. ---")

All prerequisite functions seem to be defined.

--- File Upload for Actual Run ---
Please upload your actual instance files.
Ensure they are named according to the prefix expected by 'load_and_preprocess_data'.
Enter the common prefix for your instance files (e.g., 'instance', 'problemX'): instance
Found existing file: /content/instance_coordinates.txt. Removing it before upload...
Found existing file: /content/instance_demand.txt. Removing it before upload...
Found existing file: /content/instance_service_time.txt. Removing it before upload...
Found existing file: /content/instance_time_windows.txt. Removing it before upload...


Saving instance_coordinates.txt to instance_coordinates.txt
Saving instance_demand.txt to instance_demand.txt
Saving instance_service_time.txt to instance_service_time.txt
Saving instance_time_windows.txt to instance_time_windows.txt
User uploaded file "instance_coordinates.txt" with length 3107 bytes
User uploaded file "instance_demand.txt" with length 1697 bytes
User uploaded file "instance_service_time.txt" with length 1293 bytes
User uploaded file "instance_time_windows.txt" with length 3833 bytes

All required files for the actual run seem to be uploaded.
Proceeding with the solver execution...
Enter the DEPOT ID for instance 'instance' (e.g., 0 or 1): 1
Enter the VEHICLE CAPACITY for instance 'instance' (e.g., 12600): 12600
Enter the vehicle SPEED (distance units per HOUR) for instance 'instance' (e.g., 1000): 1000
Enter the ALPHA PENALTY (cost per vehicle) for instance 'instance' (e.g., 1000, 5000): 1000
Enter the MAXIMUM Column Generation ITERATIONS for instance 'instance' (e.g